In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler,StandardScaler
import torch
from sklearn.model_selection import train_test_split
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import math
import random

# Prepare Data

In [2]:
df_no = pd.read_csv('no_discount.csv')
df_all = pd.read_csv('all_discount.csv')
df_half = pd.read_csv('half_discount.csv')
df_prod = pd.read_csv('product.csv')

In [3]:
# Map user features to number
dataframes = {'all': df_all, 'half': df_half, 'no': df_no}
feature_cols = ['user_feature_1', 'user_feature_2', 'user_feature_3']
category_maps = {}

# 1. Build mapping for each feature across all dataframes
for col in feature_cols:
    # Collect unique values from all dataframes for this column
    unique_vals = pd.concat([df[col] for df in dataframes.values()]).unique()
    cat_map = {cat: i+1 for i, cat in enumerate(sorted(unique_vals))}
    category_maps[col] = cat_map

# 2. Apply the mapping to each dataframe
for df in dataframes.values():
    for col in feature_cols:
        df[col + '_num'] = df[col].map(category_maps[col])

# 3. Show mapping for each feature
for col in feature_cols:
    print(f"\nMapping for {col}:")
    for cat, idx in category_maps[col].items():
        print(f"{idx}: {cat}")


Mapping for user_feature_1:
1: Female
2: Male
3: Non-binary / third gender
4: Prefer not to say

Mapping for user_feature_2:
1: 1-2 years ago
2: 2-3 years ago
3: 3-4 yeas ago
4: Currently, I don't have a phone
5: Less than 1 year ago
6: More than 4 years ago

Mapping for user_feature_3:
1: Brand reputation
2: Design and appearance
3: Good price
4: Product quality


In [4]:
# Map choice to number (no discount)
# Define the mapping for 'choice'
choice_map = {
    "Do not purchase; save the $30 or use it for other expenses ($30 is roughly enough to cover two meals at a fast-food restaurant in the U.S.)": 0,
    "Color: Black; Style: Solid color; Weight: 2.66 pounds; Price: $24 (MSRP: $24)": 1,
    "Color: Black; Style: Gradient color; Weight: 2.66 pounds; Price: $26 (MSRP: $26)": 2,
    "Color: Dark blue; Style: Solid color; Weight: 2.61 pounds; Price: $25 (MSRP: $25)": 3,
    "Color: Dark blue; Style: Gradient color; Weight: 2.64 pounds; Price: $25 (MSRP: $25)": 4,
    "Color: Light blue; Style: Gradient color; Weight: 2.68 pounds; Price: $26 (MSRP: $26)": 5,
    "Color: White; Style: Solid color; Weight: 2.68 pounds; Price: $27 (MSRP: $27)": 6
}

# Apply the mapping
df_no['choice_num'] = df_no['choice'].map(choice_map)

In [5]:
# Map choice to number (all discount)
# Define the mapping for 'choice'
choice_map = {
    "Do not purchase; save the $30 or use it for other expenses ($30 is roughly enough to cover two meals at a fast-food restaurant in the U.S.)": 0,
    "Color: Black; Style: Solid color; Weight: 2.66 pounds; Price: $4.8 (MSRP: $24)": 1,
    "Color: Black; Style: Gradient color; Weight: 2.66 pounds; Price: $5.2 (MSRP: $26)": 2,
    "Color: Dark blue; Style: Solid color; Weight: 2.61 pounds; Price: $5 (MSRP: $25)": 3,
    "Color: Dark blue; Style: Gradient color; Weight: 2.64 pounds; Price: $5 (MSRP: $25)": 4,
    "Color: Light blue; Style: Gradient color; Weight: 2.68 pounds; Price: $5.2 (MSRP: $26)": 5,
    "Color: White; Style: Solid color; Weight: 2.68 pounds; Price: $5.4 (MSRP: $27)": 6
}

# Apply the mapping
df_all['choice_num'] = df_all['choice'].map(choice_map)

In [6]:
# Map choice to number (half discount)
# Define the mapping for 'choice'
choice_map = {
    "Do not purchase; save the $30 or use it for other expenses ($30 is roughly enough to cover two meals at a fast-food restaurant in the U.S.)": 0,
    "Color: Black; Style: Solid color; Weight: 2.66 pounds; Price: $4.8 (MSRP: $24)": 1,
    "Color: Black; Style: Gradient color; Weight: 2.66 pounds; Price: $26 (MSRP: $26)": 2,
    "Color: Dark blue; Style: Solid color; Weight: 2.61 pounds; Price: $25 (MSRP: $25)": 3,
    "Color: Dark blue; Style: Gradient color; Weight: 2.64 pounds; Price: $25 (MSRP: $25)": 4,
    "Color: Light blue; Style: Gradient color; Weight: 2.68 pounds; Price: $5.2 (MSRP: $26)": 5,
    "Color: White; Style: Solid color; Weight: 2.68 pounds; Price: $5.4 (MSRP: $27)": 6
}

# Apply the mapping
df_half['choice_num'] = df_half['choice'].map(choice_map)

In [7]:
# Normalization:  price
# Define the columns to scale
columns_to_scale = ['price']

# Initialize the scaler
scaler = MinMaxScaler()

# Fit and transform the selected columns
df_prod[columns_to_scale] = scaler.fit_transform(df_prod[columns_to_scale])

In [8]:
# Set parameters
discount_percentage = 0.2
num_product = 6

In [9]:
# Check the number of valid data
num_row_no = len(df_no)
num_row_all = len(df_all)
num_row_half = len(df_half)
print(num_row_no)
print(num_row_all)
print(num_row_half)

291
278
266


# True GTE

In [10]:
# All products are control (no discount)
# Merge the purchasing data with the product data
merged_no = df_no.merge(df_prod, left_on='choice_num', right_on='product_id', how='left')

# Calculate the total revenue
revenue_control = 0
revenue_control = merged_no['price'].sum() * 300/291

# All products are treated (all discount)
# Merge the purchasing data with the product data
merged_all = df_all.merge(df_prod, left_on='choice_num', right_on='product_id', how='left')

# Calculate the total revenue
revenue_treated = 0
revenue_treated = merged_all['price'].sum() * discount_percentage * 300/278

# Calculate the GTE
true = revenue_treated - revenue_control
print(true)

-68.6551789496238


# Prepare Function

## DIM

In [11]:
def calculate_total_revenue(df):
    revenue_treated = 0.0
    revenue_control = 0.0
    df['adjusted_price'] = df.apply(
        lambda row: row['price'] * discount_percentage if row['if_treated'] == 1 else row['price'], axis=1
    )
    for _, row in df.iterrows():
        if row['if_treated'] == 0:
            revenue_control += row['adjusted_price']
        elif row['if_treated'] == 1:
            revenue_treated += row['adjusted_price']
    return revenue_control, revenue_treated

## MNL

In [12]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [13]:
def prepare_data_bootstrap(prod, df):
    """
    Prepare data from a dataframe (can be full dataset or bootstrap sample)
    """
    # Product features (skip the first row)
    feature_cols = ['product_feature_1', 'product_feature_2', 'product_feature_3']
    X_product = torch.tensor(
        prod[feature_cols].iloc[1:].to_numpy(),
        dtype=torch.float
    )

    # Price (skip first row)
    price = torch.tensor(
        prod['price'].iloc[1:].to_numpy(),
        dtype=torch.float
    )

    # Product randomization (skip first row)
    prod_randomization = torch.tensor(
        prod['if_treated'].iloc[1:].to_numpy(),
        dtype=torch.bool
    )

    # User features
    user_num_cols = ['user_feature_1_num', 'user_feature_2_num', 'user_feature_3_num']
    X_user = torch.tensor(
        df[user_num_cols].to_numpy(),
        dtype=torch.float
    )

    # User choice
    choice = torch.tensor(
        df['choice_num'].to_numpy(),
        dtype=torch.long
    )

    X_user = X_user.to(device)
    X_product = X_product.to(device)
    price = price.to(device)
    prod_randomization = prod_randomization.to(device)
    choice = choice.to(device)

    return X_user, choice, X_product, price, prod_randomization

In [14]:
def set_seed(seed: int):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [15]:
def init_weights(m):
    if isinstance(m, nn.Linear):
        nn.init.xavier_uniform_(m.weight)
        if m.bias is not None:
            m.bias.data.fill_(0.01)

In [16]:
def init_weights(m):
    if isinstance(m, nn.Linear):
        nn.init.xavier_uniform_(m.weight)
        if m.bias is not None:
            m.bias.data.fill_(0.01)

In [17]:
class LinearMNLModel_No(nn.Module):
    def __init__(self):
        super(LinearMNLModel_No, self).__init__()
        # v is the same constant value for all products.
        self.v = nn.Parameter(torch.tensor(1.0))
        # gamma is the price sensitivity parameter.
        self.gamma = nn.Parameter(torch.tensor(-1.0))

    def forward(self, x_user, X_product, price, prod_randomization):
        N = x_user.shape[0]
        # Adjust prices for treated products (discounted)
        adjusted_price = torch.where(
            prod_randomization,
            price * discount_percentage,
            price
        )
        # Compute utilities: shape [M]
        utility = self.v + self.gamma * adjusted_price
        # Expand to shape [N, M]
        utility = utility.expand(N, -1)
        # Add outside option (utility = 0)
        zero_utilities = torch.zeros(N, 1, device=utility.device)
        utilities_with_outside = torch.cat((zero_utilities, utility), dim=1)
        return utilities_with_outside

In [18]:
class LinearMNLModel_UserOnly(nn.Module):
    def __init__(self, user_feature_dim):
        super(LinearMNLModel_UserOnly, self).__init__()
        self.beta_user = nn.Parameter(torch.randn(user_feature_dim))
        self.beta_price = nn.Parameter(torch.tensor(-1.0))

    def forward(self, x_user, X_product, price, prod_randomization):
        N, M = x_user.shape[0], X_product.shape[0]

        # Expand user features
        x_user_expanded = x_user.unsqueeze(1).expand(-1, M, -1).detach()
        utility_user = torch.sum(x_user_expanded * self.beta_user, dim=2)

        # Price adjustment
        adjusted_price = torch.where(
            prod_randomization.unsqueeze(0),
            price * discount_percentage,
            price
        )
        utility_price = adjusted_price * self.beta_price
        utility_price = utility_price.expand(N, M)

        total_utility = utility_user + utility_price

        # Outside option
        zero_utilities = torch.zeros(N, 1, device=total_utility.device)
        utilities_with_outside = torch.cat((zero_utilities,total_utility), dim=1)
        return utilities_with_outside

In [19]:
class LinearMNLModel_ProductOnly(nn.Module):
    def __init__(self, product_feature_dim):
        super(LinearMNLModel_ProductOnly, self).__init__()
        self.beta_product = nn.Parameter(torch.randn(product_feature_dim))
        self.beta_price = nn.Parameter(torch.tensor(-1.0))

    def forward(self, x_user, X_product, price, prod_randomization):
        N, M = x_user.shape[0], X_product.shape[0]

        # Expand product features
        X_product_expanded = X_product.unsqueeze(0).expand(N, -1, -1).detach()
        utility_product = torch.sum(X_product_expanded * self.beta_product, dim=2)

        # Price adjustment
        adjusted_price = torch.where(
            prod_randomization.unsqueeze(0),
            price * discount_percentage,
            price
        )
        utility_price = adjusted_price * self.beta_price
        utility_price = utility_price.expand(N, M)

        total_utility = utility_product + utility_price

        # Outside option
        zero_utilities = torch.zeros(N, 1, device=total_utility.device)
        utilities_with_outside = torch.cat((zero_utilities,total_utility), dim=1)
        return utilities_with_outside

In [20]:
# MNL choice model
class LinearMNLModel(nn.Module):
    def __init__(self, user_feature_dim, product_feature_dim):
        super(LinearMNLModel, self).__init__()
        # Initialize parameters for user and product features
        self.beta_user = nn.Parameter(torch.randn(user_feature_dim))
        self.beta_product = nn.Parameter(torch.randn(product_feature_dim))
        self.beta_price = nn.Parameter(torch.tensor(-1.0))
    def forward(self, x_user, X_product, price, prod_randomization):
        N, M = x_user.shape[0], X_product.shape[0]

        # Expand user and product features to create a [N, M, F] shaped tensor for each
        x_user_expanded = x_user.unsqueeze(1).expand(-1, M, -1).detach()
        X_product_expanded = X_product.unsqueeze(0).expand(N, -1, -1).detach()

        # Calculate linear utility from features
        utility_user = torch.sum(x_user_expanded * self.beta_user, dim=2)
        utility_product = torch.sum(X_product_expanded * self.beta_product, dim=2)

        # Adjust prices based on randomization
        adjusted_price = torch.where(
            prod_randomization.unsqueeze(0),
            price * discount_percentage,
            price
        )

        # Calculate utility from price, properly expanding its dimension
        utility_price = adjusted_price * self.beta_price  # [M]
        utility_price = utility_price.expand(N, M)  # [N, M]

        # Total utility including features and price
        total_utility = utility_user + utility_product + utility_price

        # Incorporate the outside option with utility 0
        zero_utilities = torch.zeros(N, 1, device=total_utility.device)
        utilities_with_outside = torch.cat((zero_utilities,total_utility), dim=1)

        return utilities_with_outside

In [21]:
# Helper function to check loss drop
def loss_drop(hist, window=5):
    if len(hist) < 2 * window:
        return 0.0
    early = np.mean(hist[:window])
    late = np.mean(hist[-window:])
    return early - late

In [22]:
#Function to train model and compute revenue difference
# Returns: (linear_pe, linear_ape, trained_ok)
def train_and_estimate_linear_no(X_user, X_user_train1, X_user_test, choice_train1, choice_test, X_user_val, choice_val, X_product, price, prod_randomization, device, num_epochs=1000, lr=0.01, step_size=2500, gamma=0.5, patience=10, attempt=0,
    ci_alpha=0.05, return_ci=True):
    # Initialize the model
    model = LinearMNLModel_No().to(device)
    model.apply(init_weights)

    optimizer = optim.Adam(model.parameters(), lr)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size, gamma)

    best_val_loss = float('inf')
    best_epoch = -1
    patience_counter = 0
    trained_ok = False
    train_hist, val_hist = [], []
    best_path = f'best_model_attempt_linear_no{attempt}.pth'

    K = None  # choice set size (including outside)

    # Train the model
    for epoch in range(num_epochs):
        model.train()
        optimizer.zero_grad()

        # Forward pass (Training)
        utilities = model(X_user_train1, X_product, price, prod_randomization)
        
        if epoch == 0:
            K = utilities.shape[1]
        
        choice_probabilities = F.log_softmax(utilities, dim=1)
        loss = -torch.mean(choice_probabilities[torch.arange(choice_probabilities.shape[0]), choice_train1])


        # Numerical stability check
        if not torch.isfinite(loss).all():
            print(f"[attempt={attempt}] NaN/Inf loss at epoch {epoch}. Aborting.")
            return None, None, False

        # Backward pass
        loss.backward()
        optimizer.step()
        scheduler.step()

        model.eval()
        with torch.no_grad():
            val_utilities = model(X_user_val, X_product, price, prod_randomization)
            val_utilities = val_utilities - val_utilities.max(dim=1, keepdim=True).values
            val_choice_probabilities = F.log_softmax(val_utilities, dim=1)
            val_loss = -torch.mean(val_choice_probabilities[torch.arange(val_choice_probabilities.shape[0]), choice_val])

        train_hist.append(loss.item())
        val_hist.append(val_loss.item())

        # Early stopping
        if val_loss.item() < best_val_loss:
            best_val_loss = val_loss.item()
            best_epoch = epoch
            patience_counter = 0
            torch.save(model.state_dict(), best_path)
            trained_ok = True
        else:
            patience_counter += 1

        if patience_counter >= patience:
            print(f"[attempt={attempt}] Early stopping at epoch {epoch+1} (best epoch {best_epoch}, best val {best_val_loss:.4f})")
            break

    # Load best model
    if trained_ok:
        model.load_state_dict(torch.load(best_path))
    else:
        print(f"[attempt={attempt}] WARNING: no validation improvement.")
        return None, None, False

    # Check if training was meaningful
    drop_train = loss_drop(train_hist)
    drop_val = loss_drop(val_hist)
    if (drop_train < 1e-3) or (drop_val < 1e-3):
        print(f"[attempt={attempt}] Skipping — no meaningful training (Δtrain={drop_train:.3g}, Δval={drop_val:.3g}).")
        return None, None, False

    # Check near-random model
    if K is not None and best_val_loss > (math.log(K) - 0.05):
        print(f"[attempt={attempt}] Mark UNSTABLE: near_random, best_val={best_val_loss:.4f}, lnK={math.log(K):.4f}")
        return None, None, False

    # =========================
    # Evaluate expected revenue + CI
    # =========================
    N_products = X_product.shape[0]
    price_with_outside = torch.cat((torch.zeros(1, device=price.device), price), dim=0)  # [K], K=J+1

    # Control
    all_control = torch.zeros(N_products, dtype=torch.bool, device=device)
    utilities_control = model(X_user_test, X_product, price, all_control)
    probs_control = F.softmax(utilities_control, dim=1)  # [n_test, K]
    rev_control_i = torch.sum(
        probs_control * price_with_outside.unsqueeze(0).expand_as(probs_control),
        dim=1
    )  # [n_test]

    # Treated
    all_treated = torch.ones(N_products, dtype=torch.bool, device=device)
    utilities_treated = model(X_user_test, X_product, price, all_treated)
    probs_treated = F.softmax(utilities_treated, dim=1)
    rev_treated_i = torch.sum(
        probs_treated * price_with_outside.unsqueeze(0).expand_as(probs_treated),
        dim=1
    ) * float(discount_percentage)  # [n_test]

    diff_i = rev_treated_i - rev_control_i  # [n_test]

    n_test = diff_i.shape[0]
    scale = 300.0 / float(n_test)

    linear = (diff_i.sum() * scale).item()  # sum(diff_i)*300/n_test

    # --- CI (CLT over users) ---
    diff_sd = diff_i.std(unbiased=True)
    se_linear = (diff_sd / math.sqrt(float(n_test)) * 300.0).item()

    z = torch.distributions.Normal(0.0, 1.0).icdf(torch.tensor(1 - ci_alpha / 2)).item()
    ci_low = linear - z * se_linear
    ci_high = linear + z * se_linear

    covered = None
    if true is not None:
        covered = (ci_low <= float(true) <= ci_high)

    # --- Your PE/APE ---
    if true is None:
        linear_pe, linear_ape = None, None
    else:
        linear_pe = (linear - float(true)) / float(true)
        linear_ape = abs(linear_pe)

    if return_ci:
        print(
            f"[attempt={attempt}] est={linear:.6f}, SE={se_linear:.6f}, "
            f"{int((1-ci_alpha)*100)}% CI=[{ci_low:.6f}, {ci_high:.6f}], "
            f"covered={covered}"
        )
        return linear_pe, linear_ape, (ci_low, ci_high), covered, True

    return linear_pe, linear_ape, True

In [23]:
#Function to train model and compute revenue difference
# Returns: (linear_pe, linear_ape, trained_ok)
def train_and_estimate_linear_user(X_user, X_user_train1, X_user_test, choice_train1, choice_test, X_user_val, choice_val, X_product, price, prod_randomization, device, num_epochs=1000, lr=0.01, step_size=2500, gamma=0.5, patience=10, attempt=0,
    ci_alpha=0.05, return_ci=True):
    # Initialize the model
    model = LinearMNLModel_UserOnly(user_feature_dim=X_user.shape[1]).to(device)
    model.apply(init_weights)

    optimizer = optim.Adam(model.parameters(), lr)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size, gamma)

    best_val_loss = float('inf')
    best_epoch = -1
    patience_counter = 0
    trained_ok = False
    train_hist, val_hist = [], []
    best_path = f'best_model_attempt_linear_user{attempt}.pth'

    K = None  # choice set size (including outside)

    # Train the model
    for epoch in range(num_epochs):
        model.train()
        optimizer.zero_grad()

        # Forward pass (Training)
        utilities = model(X_user_train1, X_product, price, prod_randomization)
        
        if epoch == 0:
            K = utilities.shape[1]
        
        choice_probabilities = F.log_softmax(utilities, dim=1)
        loss = -torch.mean(choice_probabilities[torch.arange(choice_probabilities.shape[0]), choice_train1])


        # Numerical stability check
        if not torch.isfinite(loss).all():
            print(f"[attempt={attempt}] NaN/Inf loss at epoch {epoch}. Aborting.")
            return None, None, False

        # Backward pass
        loss.backward()
        optimizer.step()
        scheduler.step()

        model.eval()
        with torch.no_grad():
            val_utilities = model(X_user_val, X_product, price, prod_randomization)
            val_utilities = val_utilities - val_utilities.max(dim=1, keepdim=True).values
            val_choice_probabilities = F.log_softmax(val_utilities, dim=1)
            val_loss = -torch.mean(val_choice_probabilities[torch.arange(val_choice_probabilities.shape[0]), choice_val])

        train_hist.append(loss.item())
        val_hist.append(val_loss.item())

        # Early stopping
        if val_loss.item() < best_val_loss:
            best_val_loss = val_loss.item()
            best_epoch = epoch
            patience_counter = 0
            torch.save(model.state_dict(), best_path)
            trained_ok = True
        else:
            patience_counter += 1

        if patience_counter >= patience:
            print(f"[attempt={attempt}] Early stopping at epoch {epoch+1} (best epoch {best_epoch}, best val {best_val_loss:.4f})")
            break

    # Load best model
    if trained_ok:
        model.load_state_dict(torch.load(best_path))
    else:
        print(f"[attempt={attempt}] WARNING: no validation improvement.")
        return None, None, False

    # Check if training was meaningful
    drop_train = loss_drop(train_hist)
    drop_val = loss_drop(val_hist)
    if (drop_train < 1e-3) or (drop_val < 1e-3):
        print(f"[attempt={attempt}] Skipping — no meaningful training (Δtrain={drop_train:.3g}, Δval={drop_val:.3g}).")
        return None, None, False

    # Check near-random model
    if K is not None and best_val_loss > (math.log(K) - 0.05):
        print(f"[attempt={attempt}] Mark UNSTABLE: near_random, best_val={best_val_loss:.4f}, lnK={math.log(K):.4f}")
        return None, None, False

    # =========================
    # Evaluate expected revenue + CI
    # =========================
    N_products = X_product.shape[0]
    price_with_outside = torch.cat((torch.zeros(1, device=price.device), price), dim=0)  # [K], K=J+1

    # Control
    all_control = torch.zeros(N_products, dtype=torch.bool, device=device)
    utilities_control = model(X_user_test, X_product, price, all_control)
    probs_control = F.softmax(utilities_control, dim=1)  # [n_test, K]
    rev_control_i = torch.sum(
        probs_control * price_with_outside.unsqueeze(0).expand_as(probs_control),
        dim=1
    )  # [n_test]

    # Treated
    all_treated = torch.ones(N_products, dtype=torch.bool, device=device)
    utilities_treated = model(X_user_test, X_product, price, all_treated)
    probs_treated = F.softmax(utilities_treated, dim=1)
    rev_treated_i = torch.sum(
        probs_treated * price_with_outside.unsqueeze(0).expand_as(probs_treated),
        dim=1
    ) * float(discount_percentage)  # [n_test]

    diff_i = rev_treated_i - rev_control_i  # [n_test]

    n_test = diff_i.shape[0]
    scale = 300.0 / float(n_test)

    linear = (diff_i.sum() * scale).item()  # sum(diff_i)*300/n_test

    # --- CI (CLT over users) ---
    diff_sd = diff_i.std(unbiased=True)
    se_linear = (diff_sd / math.sqrt(float(n_test)) * 300.0).item()

    z = torch.distributions.Normal(0.0, 1.0).icdf(torch.tensor(1 - ci_alpha / 2)).item()
    ci_low = linear - z * se_linear
    ci_high = linear + z * se_linear

    covered = None
    if true is not None:
        covered = (ci_low <= float(true) <= ci_high)

    # --- Your PE/APE ---
    if true is None:
        linear_pe, linear_ape = None, None
    else:
        linear_pe = (linear - float(true)) / float(true)
        linear_ape = abs(linear_pe)

    if return_ci:
        print(
            f"[attempt={attempt}] est={linear:.6f}, SE={se_linear:.6f}, "
            f"{int((1-ci_alpha)*100)}% CI=[{ci_low:.6f}, {ci_high:.6f}], "
            f"covered={covered}"
        )
        return linear_pe, linear_ape, (ci_low, ci_high), covered, True

    return linear_pe, linear_ape, True

In [24]:
#Function to train model and compute revenue difference
# Returns: (linear_pe, linear_ape, trained_ok)
def train_and_estimate_linear_product(X_user, X_user_train1, X_user_test, choice_train1, choice_test, X_user_val, choice_val, X_product, price, prod_randomization, device, num_epochs=1000, lr=0.01, step_size=2500, gamma=0.5, patience=10, attempt=0,
    ci_alpha=0.05, return_ci=True):
    # Initialize the model
    model = LinearMNLModel_ProductOnly(product_feature_dim=X_product.shape[1]).to(device)
    model.apply(init_weights)

    optimizer = optim.Adam(model.parameters(), lr)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size, gamma)

    best_val_loss = float('inf')
    best_epoch = -1
    patience_counter = 0
    trained_ok = False
    train_hist, val_hist = [], []
    best_path = f'best_model_attempt_linear_product{attempt}.pth'

    K = None  # choice set size (including outside)

    # Train the model
    for epoch in range(num_epochs):
        model.train()
        optimizer.zero_grad()

        # Forward pass (Training)
        utilities = model(X_user_train1, X_product, price, prod_randomization)
        
        if epoch == 0:
            K = utilities.shape[1]
        
        choice_probabilities = F.log_softmax(utilities, dim=1)
        loss = -torch.mean(choice_probabilities[torch.arange(choice_probabilities.shape[0]), choice_train1])


        # Numerical stability check
        if not torch.isfinite(loss).all():
            print(f"[attempt={attempt}] NaN/Inf loss at epoch {epoch}. Aborting.")
            return None, None, False

        # Backward pass
        loss.backward()
        optimizer.step()
        scheduler.step()

        model.eval()
        with torch.no_grad():
            val_utilities = model(X_user_val, X_product, price, prod_randomization)
            val_utilities = val_utilities - val_utilities.max(dim=1, keepdim=True).values
            val_choice_probabilities = F.log_softmax(val_utilities, dim=1)
            val_loss = -torch.mean(val_choice_probabilities[torch.arange(val_choice_probabilities.shape[0]), choice_val])

        train_hist.append(loss.item())
        val_hist.append(val_loss.item())

        # Early stopping
        if val_loss.item() < best_val_loss:
            best_val_loss = val_loss.item()
            best_epoch = epoch
            patience_counter = 0
            torch.save(model.state_dict(), best_path)
            trained_ok = True
        else:
            patience_counter += 1

        if patience_counter >= patience:
            print(f"[attempt={attempt}] Early stopping at epoch {epoch+1} (best epoch {best_epoch}, best val {best_val_loss:.4f})")
            break

    # Load best model
    if trained_ok:
        model.load_state_dict(torch.load(best_path))
    else:
        print(f"[attempt={attempt}] WARNING: no validation improvement.")
        return None, None, False

    # Check if training was meaningful
    drop_train = loss_drop(train_hist)
    drop_val = loss_drop(val_hist)
    if (drop_train < 1e-3) or (drop_val < 1e-3):
        print(f"[attempt={attempt}] Skipping — no meaningful training (Δtrain={drop_train:.3g}, Δval={drop_val:.3g}).")
        return None, None, False

    # Check near-random model
    if K is not None and best_val_loss > (math.log(K) - 0.05):
        print(f"[attempt={attempt}] Mark UNSTABLE: near_random, best_val={best_val_loss:.4f}, lnK={math.log(K):.4f}")
        return None, None, False

    # =========================
    # Evaluate expected revenue + CI
    # =========================
    N_products = X_product.shape[0]
    price_with_outside = torch.cat((torch.zeros(1, device=price.device), price), dim=0)  # [K], K=J+1

    # Control
    all_control = torch.zeros(N_products, dtype=torch.bool, device=device)
    utilities_control = model(X_user_test, X_product, price, all_control)
    probs_control = F.softmax(utilities_control, dim=1)  # [n_test, K]
    rev_control_i = torch.sum(
        probs_control * price_with_outside.unsqueeze(0).expand_as(probs_control),
        dim=1
    )  # [n_test]

    # Treated
    all_treated = torch.ones(N_products, dtype=torch.bool, device=device)
    utilities_treated = model(X_user_test, X_product, price, all_treated)
    probs_treated = F.softmax(utilities_treated, dim=1)
    rev_treated_i = torch.sum(
        probs_treated * price_with_outside.unsqueeze(0).expand_as(probs_treated),
        dim=1
    ) * float(discount_percentage)  # [n_test]

    diff_i = rev_treated_i - rev_control_i  # [n_test]

    n_test = diff_i.shape[0]
    scale = 300.0 / float(n_test)

    linear = (diff_i.sum() * scale).item()  # sum(diff_i)*300/n_test

    # --- CI (CLT over users) ---
    diff_sd = diff_i.std(unbiased=True)
    se_linear = (diff_sd / math.sqrt(float(n_test)) * 300.0).item()

    z = torch.distributions.Normal(0.0, 1.0).icdf(torch.tensor(1 - ci_alpha / 2)).item()
    ci_low = linear - z * se_linear
    ci_high = linear + z * se_linear

    covered = None
    if true is not None:
        covered = (ci_low <= float(true) <= ci_high)

    # --- Your PE/APE ---
    if true is None:
        linear_pe, linear_ape = None, None
    else:
        linear_pe = (linear - float(true)) / float(true)
        linear_ape = abs(linear_pe)

    if return_ci:
        print(
            f"[attempt={attempt}] est={linear:.6f}, SE={se_linear:.6f}, "
            f"{int((1-ci_alpha)*100)}% CI=[{ci_low:.6f}, {ci_high:.6f}], "
            f"covered={covered}"
        )
        return linear_pe, linear_ape, (ci_low, ci_high), covered, True

    return linear_pe, linear_ape, True

In [25]:
#Function to train model and compute revenue difference
# Returns: (linear_pe, linear_ape, trained_ok)
def train_and_estimate_linear_all(X_user, X_user_train1, X_user_test, choice_train1, choice_test, X_user_val, choice_val, X_product, price, prod_randomization, device, num_epochs=1000, lr=0.01, step_size=2500, gamma=0.5, patience=10, attempt=0,
    ci_alpha=0.05, return_ci=True):    # Initialize the model
    model = LinearMNLModel(user_feature_dim=X_user.shape[1], product_feature_dim=X_product.shape[1]).to(device)
    model.apply(init_weights)

    optimizer = optim.Adam(model.parameters(), lr)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size, gamma)

    best_val_loss = float('inf')
    best_epoch = -1
    patience_counter = 0
    trained_ok = False
    train_hist, val_hist = [], []
    best_path = f'best_model_attempt_linear_all{attempt}.pth'

    K = None  # choice set size (including outside)

    # Train the model
    for epoch in range(num_epochs):
        model.train()
        optimizer.zero_grad()

        # Forward pass (Training)
        utilities = model(X_user_train1, X_product, price, prod_randomization)
        
        if epoch == 0:
            K = utilities.shape[1]
        
        choice_probabilities = F.log_softmax(utilities, dim=1)
        loss = -torch.mean(choice_probabilities[torch.arange(choice_probabilities.shape[0]), choice_train1])


        # Numerical stability check
        if not torch.isfinite(loss).all():
            print(f"[attempt={attempt}] NaN/Inf loss at epoch {epoch}. Aborting.")
            return None, None, False

        # Backward pass
        loss.backward()
        optimizer.step()
        scheduler.step()

        model.eval()
        with torch.no_grad():
            val_utilities = model(X_user_val, X_product, price, prod_randomization)
            val_utilities = val_utilities - val_utilities.max(dim=1, keepdim=True).values
            val_choice_probabilities = F.log_softmax(val_utilities, dim=1)
            val_loss = -torch.mean(val_choice_probabilities[torch.arange(val_choice_probabilities.shape[0]), choice_val])

        train_hist.append(loss.item())
        val_hist.append(val_loss.item())

        # Early stopping
        if val_loss.item() < best_val_loss:
            best_val_loss = val_loss.item()
            best_epoch = epoch
            patience_counter = 0
            torch.save(model.state_dict(), best_path)
            trained_ok = True
        else:
            patience_counter += 1

        if patience_counter >= patience:
            print(f"[attempt={attempt}] Early stopping at epoch {epoch+1} (best epoch {best_epoch}, best val {best_val_loss:.4f})")
            break

    # Load best model
    if trained_ok:
        model.load_state_dict(torch.load(best_path))
    else:
        print(f"[attempt={attempt}] WARNING: no validation improvement.")
        return None, None, False

    # Check if training was meaningful
    drop_train = loss_drop(train_hist)
    drop_val = loss_drop(val_hist)
    if (drop_train < 1e-3) or (drop_val < 1e-3):
        print(f"[attempt={attempt}] Skipping — no meaningful training (Δtrain={drop_train:.3g}, Δval={drop_val:.3g}).")
        return None, None, False

    # Check near-random model
    if K is not None and best_val_loss > (math.log(K) - 0.05):
        print(f"[attempt={attempt}] Mark UNSTABLE: near_random, best_val={best_val_loss:.4f}, lnK={math.log(K):.4f}")
        return None, None, False

    # =========================
    # Evaluate expected revenue + CI
    # =========================
    N_products = X_product.shape[0]
    price_with_outside = torch.cat((torch.zeros(1, device=price.device), price), dim=0)  # [K], K=J+1

    # Control
    all_control = torch.zeros(N_products, dtype=torch.bool, device=device)
    utilities_control = model(X_user_test, X_product, price, all_control)
    probs_control = F.softmax(utilities_control, dim=1)  # [n_test, K]
    rev_control_i = torch.sum(
        probs_control * price_with_outside.unsqueeze(0).expand_as(probs_control),
        dim=1
    )  # [n_test]

    # Treated
    all_treated = torch.ones(N_products, dtype=torch.bool, device=device)
    utilities_treated = model(X_user_test, X_product, price, all_treated)
    probs_treated = F.softmax(utilities_treated, dim=1)
    rev_treated_i = torch.sum(
        probs_treated * price_with_outside.unsqueeze(0).expand_as(probs_treated),
        dim=1
    ) * float(discount_percentage)  # [n_test]

    diff_i = rev_treated_i - rev_control_i  # [n_test]

    n_test = diff_i.shape[0]
    scale = 300.0 / float(n_test)

    linear = (diff_i.sum() * scale).item()  # sum(diff_i)*300/n_test

    # --- CI (CLT over users) ---
    diff_sd = diff_i.std(unbiased=True)
    se_linear = (diff_sd / math.sqrt(float(n_test)) * 300.0).item()

    z = torch.distributions.Normal(0.0, 1.0).icdf(torch.tensor(1 - ci_alpha / 2)).item()
    ci_low = linear - z * se_linear
    ci_high = linear + z * se_linear

    covered = None
    if true is not None:
        covered = (ci_low <= float(true) <= ci_high)

    # --- Your PE/APE ---
    if true is None:
        linear_pe, linear_ape = None, None
    else:
        linear_pe = (linear - float(true)) / float(true)
        linear_ape = abs(linear_pe)

    if return_ci:
        print(
            f"[attempt={attempt}] est={linear:.6f}, SE={se_linear:.6f}, "
            f"{int((1-ci_alpha)*100)}% CI=[{ci_low:.6f}, {ci_high:.6f}], "
            f"covered={covered}"
        )
        return linear_pe, linear_ape, (ci_low, ci_high), covered, True

    return linear_pe, linear_ape, True

## PDL

In [26]:
def prepare_data(user_features, product_features, prices):
    num_products = product_features.shape[0]
    all_x_other_products = []
    all_other_prices = []
    
    for i in range(num_products):
        indices = [j for j in range(num_products) if j != i]
        other_products = product_features[indices].reshape(-1)
        other_product_prices = prices[indices]
        all_x_other_products.append(other_products)
        all_other_prices.append(other_product_prices)
        
    all_x_other_products = torch.stack(all_x_other_products, dim=0)
    all_other_prices = torch.stack(all_other_prices, dim=0)
    
    return user_features, product_features, prices, all_x_other_products, all_other_prices

In [27]:
class DeepMNLModel_No(nn.Module):
    """
    Price-only Deep MNL:
    u_ij = theta(p_j)
    Outside option utility = 0.
    """
    def __init__(self, num_product, hidden=5):
        super(DeepMNLModel_No, self).__init__()

        # theta(p_j):
        # Input:  [1]
        # Output: scalar utility
        self.theta = nn.Sequential(
        nn.Linear(1, hidden),
        nn.ReLU(),
        nn.Linear(hidden, hidden),
        nn.ReLU(),
        nn.Linear(hidden, 1)

)

    def forward(self, x_user, X_product, X_other_products, x_other_prices, price, prod_randomization):
        """
        Inputs used:
        - price: [M]  own price p_j
        - prod_randomization: [M]  treatment indicator
        """
        device = price.device
        N, M = x_user.shape[0], price.shape[0]


        # Prepare input for theta: [M, 1]
        pj_input = price.unsqueeze(1)

        # Expand to all users: [N, M, 1]
        theta_input_exp = pj_input.unsqueeze(0).expand(N, -1, -1)

        # u_ij = theta(p_j): [N, M]
        utilities = self.theta(theta_input_exp).squeeze(-1)

        # Outside option utility = 0
        zero_utilities = torch.zeros(N, 1, device=device)
        utilities_with_outside = torch.cat([zero_utilities, utilities], dim=1)

        return utilities_with_outside

In [28]:
class DeepMNLModel_UserOnly(nn.Module):
    """
    Deep MNL (user + own price only):
    u_ij = theta(x_i, p_j)
    Outside option utility = 0.
    """
    def __init__(self, user_dim, num_product, hidden=5):
        super(DeepMNLModel_UserOnly, self).__init__()

        # theta(x_i, p_j)
        # Input per (i,j): [user_dim + 1]
        self.theta = nn.Sequential(
            nn.Linear(user_dim + 1, hidden),
            nn.ReLU(),
            nn.Linear(hidden, hidden),
            nn.ReLU(),
            nn.Linear(hidden, 1)
        )

    def forward(self, x_user, X_product, X_other_products,
                x_other_prices, price, prod_randomization):

        device = price.device
        N, M = x_user.shape[0], price.shape[0]

        # p_j: [M, 1]
        pj_feat = price.unsqueeze(1)          # [M, 1]

        pj_feat_exp = pj_feat.unsqueeze(0).expand(N, -1, -1)

        x_user_exp = x_user.unsqueeze(1).expand(-1, M, -1)

        theta_input = torch.cat([x_user_exp, pj_feat_exp], dim=2)

        theta_input_flat = theta_input.reshape(N * M, -1)

        # u_ij: [N*M, 1] -> [N, M]
        utilities = self.theta(theta_input_flat).view(N, M)

        # outside option utility = 0
        zero_utilities = torch.zeros(N, 1, device=device)
        utilities_with_outside = torch.cat([zero_utilities, utilities], dim=1)

        return utilities_with_outside

In [29]:
class DeepMNLModel_ProductOnly(nn.Module):
    """
    Deep MNL (product-only):
    u_ij = theta(z_j, z_-j, p_j)
    Outside option utility = 0.
    """
    def __init__(self, product_dim, num_product, hidden=5):
        super(DeepMNLModel_ProductOnly, self).__init__()

        # Aggregate other products' features z_-j:
        # Input:  [M, (M-1)*F_p]
        # Output: [M, hidden]
        self.other_product_features_layers = nn.Sequential(
            nn.Linear(product_dim * (num_product - 1), hidden),
            nn.ReLU(),
            nn.Linear(hidden, hidden),
            nn.ReLU()
        )

        # theta(z_j, z_-j, p_j)
        # Input per (i,j): [F_p + hidden (z_-j) + 1 (p_j)]
        in_dim = product_dim + hidden + 1

        self.theta = nn.Sequential(
            nn.Linear(in_dim, hidden),
            nn.ReLU(),
            nn.Linear(hidden, hidden),
            nn.ReLU(),
            nn.Linear(hidden, 1)
        )

    def forward(self, x_user, X_product, X_other_products,
                x_other_prices, price, prod_randomization):
        """
        X_product:        [M, F_p]          product features z_j
        X_other_products: [M, (M-1)*F_p]    other products' features z_-j
        price:            [M]               own price p_j (already discounted)
        """
        device = price.device
        # N users, M products
        N, M = x_user.shape[0], X_product.shape[0]

        # z_-j features: [M, hidden]
        zminus_feat = self.other_product_features_layers(X_other_products)

        # p_j scalar: [M, 1]
        pj_feat = price.unsqueeze(1)

        # Combine product-side info: [M, F_p + hidden + 1]
        product_feat = torch.cat(
            [X_product, zminus_feat, pj_feat],
            dim=1
        )

        # Expand to all users: [N, M, in_dim]
        product_feat_exp = product_feat.unsqueeze(0).expand(N, -1, -1)

        # Flatten to [N*M, in_dim]
        theta_input_flat = product_feat_exp.reshape(N * M, -1)

        # Utilities: [N*M, 1] -> [N, M]
        utilities = self.theta(theta_input_flat).view(N, M)

        # Outside option = 0
        zero_utilities = torch.zeros(N, 1, device=device)
        utilities_with_outside = torch.cat([zero_utilities, utilities], dim=1)

        return utilities_with_outside

In [30]:
# PDL choice model
class DeepMNLModel(nn.Module):
    """
    Deep MNL:
    u_ij = theta(x_i, z_j, z_-j, p_j)
    Outside option utility = 0.
    """
    def __init__(self, user_dim, product_dim, num_product, hidden=5):
        super(DeepMNLModel, self).__init__()

        # Aggregate other products' features z_-j:
        # Input:  [M, (M-1)*F_p]
        # Output: [M, hidden]
        self.other_product_features_layers = nn.Sequential(
            nn.Linear(product_dim * (num_product - 1), hidden),
            nn.ReLU(),
            nn.Linear(hidden, hidden),
            nn.ReLU()
        )

        # theta(x_i, z_j, z_-j, p_j)
        # Input per (i,j):
        #   user_dim (x_i)
        # + product_dim (z_j)
        # + hidden (z_-j)
        # + 1 (p_j)
        in_dim = user_dim + product_dim + hidden + 1

        self.theta = nn.Sequential(
            nn.Linear(in_dim, hidden),
            nn.ReLU(),
            nn.Linear(hidden, 1)
        )

    def forward(self, x_user, X_product, X_other_products,
                x_other_prices, price, prod_randomization):
        """
        x_user:          [N, F_u]          user features x_i
        X_product:       [M, F_p]          product features z_j
        X_other_products:[M, (M-1)*F_p]    other products' features z_-j
        price:           [M]               own price p_j (already discounted)

        x_other_prices, prod_randomization are not used here.
        """
        device = price.device
        N, M = x_user.shape[0], X_product.shape[0]

        # z_-j features: [M, hidden]
        zminus_feat = self.other_product_features_layers(X_other_products)

        # p_j: [M, 1]
        pj_feat = price.unsqueeze(1)

        # Product-side block: [M, F_p + hidden + 1]
        product_block = torch.cat(
            [X_product, zminus_feat, pj_feat],
            dim=1
        )

        # Expand product block for all users: [N, M, *]
        product_block_exp = product_block.unsqueeze(0).expand(N, -1, -1)

        # Expand user features: [N, 1, F_u] -> [N, M, F_u]
        x_user_exp = x_user.unsqueeze(1).expand(-1, M, -1)

        # Final theta input: [N, M, in_dim]
        theta_input = torch.cat([x_user_exp, product_block_exp], dim=2)

        # Flatten to [N*M, in_dim]
        theta_input_flat = theta_input.reshape(N * M, -1)

        # Utilities: [N*M, 1] -> [N, M]
        utilities = self.theta(theta_input_flat).view(N, M)

        # Outside option = 0
        zero_utilities = torch.zeros(N, 1, device=device)
        utilities_with_outside = torch.cat([zero_utilities, utilities], dim=1)

        return utilities_with_outside

In [31]:
def train_and_estimate_pdl_no(X_user, X_user_train1, X_user_test, choice_train1, choice_test, X_user_val, choice_val, X_product, price, prod_randomization, device, num_epochs=1000, lr=0.01, step_size=2500, gamma=0.5, patience=10, attempt=0,
    ci_alpha=0.05, return_ci=True):    
    # --- Prepare Data ---
    prepared_data = prepare_data(X_user_train1, X_product,  price * (1 - (1-discount_percentage) * prod_randomization))
    user_features, product_features, prices_train, all_x_other_products, all_other_prices = prepared_data
    
    prepared_val = prepare_data(X_user_val, X_product, price * (1 - (1-discount_percentage) * prod_randomization))
    user_features_val, product_features_val, prices_val, all_x_other_products_val, all_other_prices_val = prepared_val

    user_features = user_features.to(device)
    product_features = product_features.to(device)
    prices_train = prices_train.to(device)
    all_x_other_products = all_x_other_products.to(device)
    all_other_prices = all_other_prices.to(device)

    user_features_val = user_features_val.to(device)
    product_features_val = product_features_val.to(device)
    prices_val = prices_val.to(device)
    all_x_other_products_val = all_x_other_products_val.to(device)
    all_other_prices_val = all_other_prices_val.to(device)
    
    # Initialize model
    model = DeepMNLModel_No(num_product=X_product.shape[0], hidden=5).to(device)

    model.apply(init_weights)

    optimizer = optim.Adam(model.parameters(), lr)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size, gamma)

    best_val_loss = float('inf')
    best_epoch = -1
    patience_counter = 0
    trained_ok = False
    train_hist, val_hist = [], []
    best_path = f'best_model_attempt_pdl_no{attempt}.pth'

    K = None 

    for epoch in range(num_epochs):
        model.train()
        optimizer.zero_grad()

        # Forward pass
        utilities = model(user_features, product_features, all_x_other_products, all_other_prices, prices_train, prod_randomization)
        
        if epoch == 0:
            K = utilities.shape[1]
        
        choice_probabilities = F.log_softmax(utilities, dim=1)
        loss = -torch.mean(choice_probabilities[torch.arange(choice_probabilities.shape[0]), choice_train1])

        if not torch.isfinite(loss).all():
            print(f"[attempt={attempt}] NaN/Inf loss at epoch {epoch}. Aborting.")
            return None, None, False

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()

        # Validation
        model.eval()
        with torch.no_grad():
            val_utilities = model(user_features_val, product_features_val, all_x_other_products_val, all_other_prices_val, prices_val, prod_randomization)
            val_utilities = val_utilities - val_utilities.max(dim=1, keepdim=True).values
            val_choice_probabilities = F.log_softmax(val_utilities, dim=1)
            val_loss = -torch.mean(val_choice_probabilities[torch.arange(val_choice_probabilities.shape[0]), choice_val])

        train_hist.append(loss.item())
        val_hist.append(val_loss.item())

        if val_loss.item() < best_val_loss:
            best_val_loss = val_loss.item()
            best_epoch = epoch
            patience_counter = 0
            torch.save(model.state_dict(), best_path)
            trained_ok = True
        else:
            patience_counter += 1

        if patience_counter >= patience:
            # print(f"[attempt={attempt}] Early stopping at epoch {epoch+1}")
            break

    if trained_ok:
        model.load_state_dict(torch.load(best_path))
    else:
        return None, None, False

    # Check logic (skipping detailed check for brevity, kept yours)
    drop_train = loss_drop(train_hist)
    drop_val = loss_drop(val_hist)
    if (drop_train < 1e-3) or (drop_val < 1e-3):
        return None, None, False

    if K is not None and best_val_loss > (math.log(K) - 0.05):
        return None, None, False

    model.eval()

    prepared_test = prepare_data(X_user_test, X_product, price)  
    user_features_test, product_features_test, prices_test, all_x_other_products_test, all_other_prices_test = prepared_test
    user_features_test = user_features_test.to(device)
    product_features_test = product_features_test.to(device)
    all_x_other_products_test = all_x_other_products_test.to(device)
    all_other_prices_test = all_other_prices_test.to(device)

    price = price.to(device)

    price_control = price
    price_treated = price * discount_percentage

    with torch.no_grad():
        u_c = model(user_features_test, product_features_test,
                    all_x_other_products_test, all_other_prices_test,
                    price_control, prod_randomization)
        u_c = u_c - u_c.max(dim=1, keepdim=True).values
        p_c = F.softmax(u_c, dim=1)

        u_t = model(user_features_test, product_features_test,
                    all_x_other_products_test, all_other_prices_test,
                    price_treated, prod_randomization)
        u_t = u_t - u_t.max(dim=1, keepdim=True).values
        p_t = F.softmax(u_t, dim=1)

    price_with_outside_c = torch.cat((torch.zeros(1, device=device), price_control), dim=0)  # [K]
    price_with_outside_t = torch.cat((torch.zeros(1, device=device), price_treated), dim=0)  # [K]

    # --- per-user expected revenue (needed for SE/CI) ---
    rev_control_i = (p_c * price_with_outside_c.unsqueeze(0)).sum(dim=1)  # [n_test]
    rev_treated_i = (p_t * price_with_outside_t.unsqueeze(0)).sum(dim=1)  # [n_test]

    diff_i = rev_treated_i - rev_control_i  # [n_test]

    n_test = diff_i.shape[0]
    scale = 300.0 / float(n_test)
    pdl = (diff_i.sum() * scale).item()  # = diff.mean()*300

    # --- CI (CLT over users) ---
    diff_sd = diff_i.std(unbiased=True)
    se_pdl = (diff_sd / math.sqrt(float(n_test)) * 300.0).item()

    z = torch.distributions.Normal(0.0, 1.0).icdf(
        torch.tensor(1 - ci_alpha / 2, device=device)
    ).item()

    ci_low = pdl - z * se_pdl
    ci_high = pdl + z * se_pdl

    covered = None
    if true is not None:
        covered = (ci_low <= float(true) <= ci_high)

    # --- PE/APE ---
    if true is None:
        pdl_pe, pdl_ape = None, None
    else:
        pdl_pe = (pdl - float(true)) / float(true)
        pdl_ape = abs(pdl_pe)

    if return_ci:
        print(
            f"[attempt={attempt}] est={pdl:.6f}, SE={se_pdl:.6f}, "
            f"{int((1-ci_alpha)*100)}% CI=[{ci_low:.6f}, {ci_high:.6f}], "
            f"covered={covered}"
        )
        return pdl_pe, pdl_ape, (ci_low, ci_high), covered, True

    return pdl_pe, pdl_ape, True


In [32]:
def train_and_estimate_pdl_user(X_user, X_user_train1, X_user_test, choice_train1, choice_test, X_user_val, choice_val, X_product, price, prod_randomization, device, num_epochs=1000, lr=0.01, step_size=2500, gamma=0.5, patience=10, attempt=0,
    ci_alpha=0.05, return_ci=True):
    # --- Prepare Data ---
    prepared_data = prepare_data(X_user_train1, X_product,  price * (1 - (1-discount_percentage) * prod_randomization))
    user_features, product_features, prices_train, all_x_other_products, all_other_prices = prepared_data
    
    prepared_val = prepare_data(X_user_val, X_product, price * (1 - (1-discount_percentage) * prod_randomization))
    user_features_val, product_features_val, prices_val, all_x_other_products_val, all_other_prices_val = prepared_val

    user_features = user_features.to(device)
    product_features = product_features.to(device)
    prices_train = prices_train.to(device)
    all_x_other_products = all_x_other_products.to(device)
    all_other_prices = all_other_prices.to(device)

    user_features_val = user_features_val.to(device)
    product_features_val = product_features_val.to(device)
    prices_val = prices_val.to(device)
    all_x_other_products_val = all_x_other_products_val.to(device)
    all_other_prices_val = all_other_prices_val.to(device)
    
    # Initialize model
    model = DeepMNLModel_UserOnly(user_dim=X_user.shape[1], num_product=X_product.shape[0], hidden=16).to(device)

    model.apply(init_weights)

    optimizer = optim.AdamW(model.parameters(), lr, weight_decay=1e-3)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size, gamma)

    best_val_loss = float('inf')
    best_epoch = -1
    patience_counter = 0
    trained_ok = False
    train_hist, val_hist = [], []
    best_path = f'best_model_attempt_pdl_user{attempt}.pth'

    K = None 

    for epoch in range(num_epochs):
        model.train()
        optimizer.zero_grad()

        # Forward pass
        utilities = model(user_features, product_features, all_x_other_products, all_other_prices, prices_train, prod_randomization)
        
        if epoch == 0:
            K = utilities.shape[1]
        
        choice_probabilities = F.log_softmax(utilities, dim=1)
        loss = -torch.mean(choice_probabilities[torch.arange(choice_probabilities.shape[0]), choice_train1])

        if not torch.isfinite(loss).all():
            print(f"[attempt={attempt}] NaN/Inf loss at epoch {epoch}. Aborting.")
            return None, None, False

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
        optimizer.step()
        scheduler.step()

        # Validation
        model.eval()
        with torch.no_grad():
            val_utilities = model(user_features_val, product_features_val, all_x_other_products_val, all_other_prices_val, prices_val, prod_randomization)
            val_utilities = val_utilities - val_utilities.max(dim=1, keepdim=True).values
            val_choice_probabilities = F.log_softmax(val_utilities, dim=1)
            val_loss = -torch.mean(val_choice_probabilities[torch.arange(val_choice_probabilities.shape[0]), choice_val])

        train_hist.append(loss.item())
        val_hist.append(val_loss.item())

        if val_loss.item() < best_val_loss:
            best_val_loss = val_loss.item()
            best_epoch = epoch
            patience_counter = 0
            torch.save(model.state_dict(), best_path)
            trained_ok = True
        else:
            patience_counter += 1

        if patience_counter >= patience:
            # print(f"[attempt={attempt}] Early stopping at epoch {epoch+1}")
            break

    if trained_ok:
        model.load_state_dict(torch.load(best_path))
    else:
        return None, None, False

    # Check logic (skipping detailed check for brevity, kept yours)
    drop_train = loss_drop(train_hist)
    drop_val = loss_drop(val_hist)
    if (drop_train < 1e-3) or (drop_val < 1e-3):
        return None, None, False

    if K is not None and best_val_loss > (math.log(K) - 0.05):
        return None, None, False

    model.eval()

    # move to device
    X_user_test = X_user_test.to(device)
    X_product = X_product.to(device)
    price = price.to(device)

    price_control = price
    price_treated = price * discount_percentage

    eval_x_other = all_x_other_products.to(device)
    eval_other_prices = all_other_prices.to(device)

    with torch.no_grad():
        # Control probs
        u_c = model(
            X_user_test, X_product,
            eval_x_other, eval_other_prices,
            price_control,
            prod_randomization
        )
        u_c = u_c - u_c.max(dim=1, keepdim=True).values
        p_c = F.softmax(u_c, dim=1)

        # Treated probs (recompute under discounted prices!)
        u_t = model(
            X_user_test, X_product,
            eval_x_other, eval_other_prices,
            price_treated,
            prod_randomization
        )
        u_t  = u_t - u_t.max(dim=1, keepdim=True).values
        p_t= F.softmax(u_t, dim=1)

    price_with_outside_c = torch.cat((torch.zeros(1, device=device), price_control), dim=0)  # [K]
    price_with_outside_t = torch.cat((torch.zeros(1, device=device), price_treated), dim=0)  # [K]

    # --- per-user expected revenue (needed for SE/CI) ---
    rev_control_i = (p_c * price_with_outside_c.unsqueeze(0)).sum(dim=1)  # [n_test]
    rev_treated_i = (p_t * price_with_outside_t.unsqueeze(0)).sum(dim=1)  # [n_test]

    diff_i = rev_treated_i - rev_control_i  # [n_test]

    n_test = diff_i.shape[0]
    scale = 300.0 / float(n_test)
    pdl = (diff_i.sum() * scale).item()  # = diff.mean()*300

    # --- CI (CLT over users) ---
    diff_sd = diff_i.std(unbiased=True)
    se_pdl = (diff_sd / math.sqrt(float(n_test)) * 300.0).item()

    z = torch.distributions.Normal(0.0, 1.0).icdf(
        torch.tensor(1 - ci_alpha / 2, device=device)
    ).item()

    ci_low = pdl - z * se_pdl
    ci_high = pdl + z * se_pdl

    covered = None
    if true is not None:
        covered = (ci_low <= float(true) <= ci_high)

    # --- PE/APE ---
    if true is None:
        pdl_pe, pdl_ape = None, None
    else:
        pdl_pe = (pdl - float(true)) / float(true)
        pdl_ape = abs(pdl_pe)

    if return_ci:
        print(
            f"[attempt={attempt}] est={pdl:.6f}, SE={se_pdl:.6f}, "
            f"{int((1-ci_alpha)*100)}% CI=[{ci_low:.6f}, {ci_high:.6f}], "
            f"covered={covered}"
        )
        return pdl_pe, pdl_ape, (ci_low, ci_high), covered, True

    return pdl_pe, pdl_ape, True

In [33]:
def train_and_estimate_pdl_product(X_user, X_user_train1, X_user_test, choice_train1, choice_test, X_user_val, choice_val, X_product, price, prod_randomization, device, num_epochs=1000, lr=0.01, step_size=2500, gamma=0.5, patience=10, attempt=0,
    ci_alpha=0.05, return_ci=True):
    # --- Prepare Data ---
    prepared_data = prepare_data(X_user_train1, X_product,  price * (1 - (1-discount_percentage) * prod_randomization))
    user_features, product_features, prices_train, all_x_other_products, all_other_prices = prepared_data
    
    prepared_val = prepare_data(X_user_val, X_product, price * (1 - (1-discount_percentage) * prod_randomization))
    user_features_val, product_features_val, prices_val, all_x_other_products_val, all_other_prices_val = prepared_val
    
    user_features = user_features.to(device)
    product_features = product_features.to(device)
    prices_train = prices_train.to(device)
    all_x_other_products = all_x_other_products.to(device)
    all_other_prices = all_other_prices.to(device)

    user_features_val = user_features_val.to(device)
    product_features_val = product_features_val.to(device)
    prices_val = prices_val.to(device)
    all_x_other_products_val = all_x_other_products_val.to(device)
    all_other_prices_val = all_other_prices_val.to(device)
    
    # Initialize model
    model = DeepMNLModel_ProductOnly(product_dim=X_product.shape[1], num_product=X_product.shape[0], hidden=16).to(device)

    model.apply(init_weights)

    optimizer = optim.AdamW(model.parameters(), lr, weight_decay=5e-4)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size, gamma)

    best_val_loss = float('inf')
    best_epoch = -1
    patience_counter = 0
    trained_ok = False
    train_hist, val_hist = [], []
    best_path = f'best_model_attempt_pdl_product{attempt}.pth'

    K = None 

    for epoch in range(num_epochs):
        model.train()
        optimizer.zero_grad()

        # Forward pass
        utilities = model(user_features, product_features, all_x_other_products, all_other_prices, prices_train, prod_randomization)
        
        if epoch == 0:
            K = utilities.shape[1]
        
        choice_probabilities = F.log_softmax(utilities, dim=1)
        loss = -torch.mean(choice_probabilities[torch.arange(choice_probabilities.shape[0]), choice_train1])

        if not torch.isfinite(loss).all():
            print(f"[attempt={attempt}] NaN/Inf loss at epoch {epoch}. Aborting.")
            return None, None, False

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()

        # Validation
        model.eval()
        with torch.no_grad():
            val_utilities = model(user_features_val, product_features_val, all_x_other_products_val, all_other_prices_val, prices_val, prod_randomization)
            val_utilities = val_utilities - val_utilities.max(dim=1, keepdim=True).values
            val_choice_probabilities = F.log_softmax(val_utilities, dim=1)
            val_loss = -torch.mean(val_choice_probabilities[torch.arange(val_choice_probabilities.shape[0]), choice_val])

        train_hist.append(loss.item())
        val_hist.append(val_loss.item())

        if val_loss.item() < best_val_loss:
            best_val_loss = val_loss.item()
            best_epoch = epoch
            patience_counter = 0
            torch.save(model.state_dict(), best_path)
            trained_ok = True
        else:
            patience_counter += 1

        if patience_counter >= patience:
            # print(f"[attempt={attempt}] Early stopping at epoch {epoch+1}")
            break

    if trained_ok:
        model.load_state_dict(torch.load(best_path))
    else:
        return None, None, False

    # Check logic (skipping detailed check for brevity, kept yours)
    drop_train = loss_drop(train_hist)
    drop_val = loss_drop(val_hist)
    if (drop_train < 1e-3) or (drop_val < 1e-3):
        return None, None, False

    if K is not None and best_val_loss > (math.log(K) - 0.05):
        return None, None, False

    # -------------------------------
    # Evaluate counterfactual revenues
    # -------------------------------
    model.eval()

    # move test tensors to device
    X_user_test = X_user_test.to(device)
    X_product = X_product.to(device)
    price = price.to(device)

    eval_x_other = all_x_other_products.to(device)
    eval_other_prices = all_other_prices.to(device)

    # counterfactual price vectors
    price_control = price
    price_treated = price * discount_percentage

    with torch.no_grad():
        # Control probs
        u_c = model(
            X_user_test, X_product,
            eval_x_other, eval_other_prices,
            price_control,
            prod_randomization
        )
        u_c = u_c - u_c.max(dim=1, keepdim=True).values
        p_c = F.softmax(u_c, dim=1)

        # Treated probs (recompute under discounted prices!)
        u_t = model(
            X_user_test, X_product,
            eval_x_other, eval_other_prices,
            price_treated,
            prod_randomization
        )
        u_t  = u_t - u_t.max(dim=1, keepdim=True).values
        p_t= F.softmax(u_t, dim=1)

    price_with_outside_c = torch.cat((torch.zeros(1, device=device), price_control), dim=0)  # [K]
    price_with_outside_t = torch.cat((torch.zeros(1, device=device), price_treated), dim=0)  # [K]

    # --- per-user expected revenue (needed for SE/CI) ---
    rev_control_i = (p_c * price_with_outside_c.unsqueeze(0)).sum(dim=1)  # [n_test]
    rev_treated_i = (p_t * price_with_outside_t.unsqueeze(0)).sum(dim=1)  # [n_test]

    diff_i = rev_treated_i - rev_control_i  # [n_test]

    n_test = diff_i.shape[0]
    scale = 300.0 / float(n_test)
    pdl = (diff_i.sum() * scale).item()  # = diff.mean()*300

    # --- CI (CLT over users) ---
    diff_sd = diff_i.std(unbiased=True)
    se_pdl = (diff_sd / math.sqrt(float(n_test)) * 300.0).item()

    z = torch.distributions.Normal(0.0, 1.0).icdf(
        torch.tensor(1 - ci_alpha / 2, device=device)
    ).item()

    ci_low = pdl - z * se_pdl
    ci_high = pdl + z * se_pdl

    covered = None
    if true is not None:
        covered = (ci_low <= float(true) <= ci_high)

    # --- PE/APE ---
    if true is None:
        pdl_pe, pdl_ape = None, None
    else:
        pdl_pe = (pdl - float(true)) / float(true)
        pdl_ape = abs(pdl_pe)

    if return_ci:
        print(
            f"[attempt={attempt}] est={pdl:.6f}, SE={se_pdl:.6f}, "
            f"{int((1-ci_alpha)*100)}% CI=[{ci_low:.6f}, {ci_high:.6f}], "
            f"covered={covered}"
        )
        return pdl_pe, pdl_ape, (ci_low, ci_high), covered, True

    return pdl_pe, pdl_ape, True


In [34]:
def train_and_estimate_pdl_all(X_user, X_user_train1, X_user_test, choice_train1, choice_test, X_user_val, choice_val, X_product, price, prod_randomization, device, num_epochs=1000, lr=0.01, step_size=2500, gamma=0.5, patience=10, attempt=0,
    ci_alpha=0.05, return_ci=True):
    # --- Prepare Data ---
    prepared_data = prepare_data(X_user_train1, X_product,  price * (1 - (1-discount_percentage) * prod_randomization))
    user_features, product_features, prices_train, all_x_other_products, all_other_prices = prepared_data
    
    prepared_val = prepare_data(X_user_val, X_product, price * (1 - (1-discount_percentage) * prod_randomization))
    user_features_val, product_features_val, prices_val, all_x_other_products_val, all_other_prices_val = prepared_val

    user_features = user_features.to(device)
    product_features = product_features.to(device)
    prices_train = prices_train.to(device)
    all_x_other_products = all_x_other_products.to(device)
    all_other_prices = all_other_prices.to(device)

    user_features_val = user_features_val.to(device)
    product_features_val = product_features_val.to(device)
    prices_val = prices_val.to(device)
    all_x_other_products_val = all_x_other_products_val.to(device)
    all_other_prices_val = all_other_prices_val.to(device)
    
    # Initialize model
    model = DeepMNLModel(user_dim=X_user.shape[1], product_dim=X_product.shape[1], num_product=X_product.shape[0], hidden=16).to(device)

    model.apply(init_weights)

    optimizer = optim.AdamW(model.parameters(), lr, weight_decay=1e-3)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size, gamma)

    best_val_loss = float('inf')
    best_epoch = -1
    patience_counter = 0
    trained_ok = False
    train_hist, val_hist = [], []
    best_path = f'best_model_attempt_pdl_all{attempt}.pth'
    l2_lambda = 1e-4

    K = None 

    for epoch in range(num_epochs):
        model.train()
        optimizer.zero_grad()

        # Forward pass
        utilities = model(user_features, product_features, all_x_other_products, all_other_prices, prices_train, prod_randomization)
        
        if epoch == 0:
            K = utilities.shape[1]
        
        choice_probabilities = F.log_softmax(utilities, dim=1)
        loss = -torch.mean(choice_probabilities[torch.arange(choice_probabilities.shape[0]), choice_train1])
        loss = loss + l2_lambda * sum(p.pow(2).sum() for p in model.parameters())
        
        if not torch.isfinite(loss).all():
            print(f"[attempt={attempt}] NaN/Inf loss at epoch {epoch}. Aborting.")
            return None, None, False

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()

        # Validation
        model.eval()
        with torch.no_grad():
            val_utilities = model(user_features_val, product_features_val, all_x_other_products_val, all_other_prices_val, prices_val, prod_randomization)
            val_utilities = val_utilities - val_utilities.max(dim=1, keepdim=True).values
            val_choice_probabilities = F.log_softmax(val_utilities, dim=1)
            val_loss = -torch.mean(val_choice_probabilities[torch.arange(val_choice_probabilities.shape[0]), choice_val])

        train_hist.append(loss.item())
        val_hist.append(val_loss.item())

        if val_loss.item() < best_val_loss:
            best_val_loss = val_loss.item()
            best_epoch = epoch
            patience_counter = 0
            torch.save(model.state_dict(), best_path)
            trained_ok = True
        else:
            patience_counter += 1

        if patience_counter >= patience:
            # print(f"[attempt={attempt}] Early stopping at epoch {epoch+1}")
            break

    if trained_ok:
        model.load_state_dict(torch.load(best_path))
    else:
        return None, None, False

    # Check logic (skipping detailed check for brevity, kept yours)
    drop_train = loss_drop(train_hist)
    drop_val = loss_drop(val_hist)
    if (drop_train < 1e-3) or (drop_val < 1e-3):
        return None, None, False

    if K is not None and best_val_loss > (math.log(K) - 0.05):
        return None, None, False
    # -------------------------------
    # Evaluate counterfactual revenues
    # -------------------------------
    model.eval()

    # move test tensors to device
    X_user_test = X_user_test.to(device)
    X_product = X_product.to(device)
    price = price.to(device)

    eval_x_other = all_x_other_products.to(device)
    eval_other_prices = all_other_prices.to(device)

    # counterfactual price vectors
    price_control = price
    price_treated = price * discount_percentage

    with torch.no_grad():
        # Control probs
        u_c = model(
            X_user_test, X_product,
            eval_x_other, eval_other_prices,
            price_control,
            prod_randomization
        )
        u_c = u_c - u_c.max(dim=1, keepdim=True).values
        p_c = F.softmax(u_c, dim=1)

        # Treated probs (recompute under discounted prices!)
        u_t = model(
            X_user_test, X_product,
            eval_x_other, eval_other_prices,
            price_treated,
            prod_randomization
        )
        u_t  = u_t - u_t.max(dim=1, keepdim=True).values
        p_t= F.softmax(u_t, dim=1)

    price_with_outside_c = torch.cat((torch.zeros(1, device=device), price_control), dim=0)  # [K]
    price_with_outside_t = torch.cat((torch.zeros(1, device=device), price_treated), dim=0)  # [K]

    # --- per-user expected revenue (needed for SE/CI) ---
    rev_control_i = (p_c * price_with_outside_c.unsqueeze(0)).sum(dim=1)  # [n_test]
    rev_treated_i = (p_t * price_with_outside_t.unsqueeze(0)).sum(dim=1)  # [n_test]

    diff_i = rev_treated_i - rev_control_i  # [n_test]

    n_test = diff_i.shape[0]
    scale = 300.0 / float(n_test)
    pdl = (diff_i.sum() * scale).item()  # = diff.mean()*300

    # --- CI (CLT over users) ---
    diff_sd = diff_i.std(unbiased=True)
    se_pdl = (diff_sd / math.sqrt(float(n_test)) * 300.0).item()

    z = torch.distributions.Normal(0.0, 1.0).icdf(
        torch.tensor(1 - ci_alpha / 2, device=device)
    ).item()

    ci_low = pdl - z * se_pdl
    ci_high = pdl + z * se_pdl

    covered = None
    if true is not None:
        covered = (ci_low <= float(true) <= ci_high)

    # --- PE/APE ---
    if true is None:
        pdl_pe, pdl_ape = None, None
    else:
        pdl_pe = (pdl - float(true)) / float(true)
        pdl_ape = abs(pdl_pe)

    if return_ci:
        print(
            f"[attempt={attempt}] est={pdl:.6f}, SE={se_pdl:.6f}, "
            f"{int((1-ci_alpha)*100)}% CI=[{ci_low:.6f}, {ci_high:.6f}], "
            f"covered={covered}"
        )
        return pdl_pe, pdl_ape, (ci_low, ci_high), covered, True

    return pdl_pe, pdl_ape, True


## DML

In [35]:
def H_theta(theta0_output, theta1_output, all_treated_price, price):
    N = theta0_output.shape[0]
    M = theta0_output.shape[1]
    device = theta0_output.device

    # Utilities
    expand_price = price.unsqueeze(0).expand(N, M)
    expand_all_treated_price = all_treated_price.unsqueeze(0).expand(N, M)
    all_treated_uti = theta0_output + theta1_output * expand_all_treated_price
    all_control_uti =  theta0_output + theta1_output * expand_price

    # Add outside option
    zero_utilities = torch.zeros(N, 1, device=device)
    all_treated_uti = torch.cat((zero_utilities, all_treated_uti), dim=1)
    all_control_uti = torch.cat((zero_utilities, all_control_uti), dim=1)

    # Probabilities
    prob_t = F.softmax(all_treated_uti, dim=1)
    prob_c = F.softmax(all_control_uti, dim=1)

    # Prices with outside option
    p_c = torch.cat((torch.zeros(1, device=device), price), dim=0)
    p_t = torch.cat((torch.zeros(1, device=device), all_treated_price), dim=0)

    # H = Expected Revenue Difference
    H = torch.sum(prob_t * p_t - prob_c * p_c, dim=1)

    # Correct Derivatives using P(1-P)
    grad_t = prob_t * (1 - prob_t)
    grad_c = prob_c * (1 - prob_c)

    H_theta0 = torch.sum(grad_t * p_t - grad_c * p_c, dim=1)
    H_theta1 = torch.sum(grad_t * (p_t**2) - grad_c * (p_c**2), dim=1)

    return H, H_theta0, H_theta1

In [36]:
def l_theta(theta0_output, theta1_output, adjusted_price, decision_test):
    N = theta0_output.shape[0]
    M = num_product
    expand_adjusted_price = adjusted_price.unsqueeze(0).expand(N, M)
    uti = theta0_output + theta1_output * expand_adjusted_price
    adjusted_price_with_outside =  torch.cat([torch.zeros(1, device=adjusted_price.device),adjusted_price])

    zero_utilities = torch.zeros(N, 1, device=uti.device)
    uti = torch.cat((zero_utilities,uti), dim=1)

    probabilities = F.softmax(uti, dim=1)
    prod_indices = torch.ones(num_product, device=device)
    prod_indices = torch.cat([torch.zeros(1,device=device),prod_indices])


    ltheta0 = probabilities[torch.arange(decision_test.size(0)), decision_test] - prod_indices[decision_test]
    ltheta1 = (probabilities[torch.arange(decision_test.size(0)), decision_test] *
           adjusted_price_with_outside[decision_test]) - (adjusted_price_with_outside[decision_test] * prod_indices[decision_test])
    return ltheta0,ltheta1

In [37]:
# l_inv
def lambdainv(theta0_output, theta1_output, price, decision_test, epsilon=1e-6):

    N = theta0_output.shape[0]
    M = num_product

    expand_price = price.unsqueeze(0).expand(N, M)
    expand_all_treated_price = discount_percentage * price.unsqueeze(0).expand(N, M)

    all_treated_uti = theta0_output + theta1_output * expand_all_treated_price
    all_control_uti = theta0_output + theta1_output * expand_price

    zero_utilities = torch.zeros(N, 1, device=all_control_uti.device)
    all_treated_uti = torch.cat((zero_utilities, all_treated_uti), dim=1)
    all_control_uti = torch.cat((zero_utilities, all_control_uti), dim=1)

    probabilities_control = F.softmax(all_control_uti, dim=1)
    probabilities_treated = F.softmax(all_treated_uti, dim=1)

    chosen_prob_control = probabilities_control[torch.arange(N), decision_test]
    chosen_prob_treated = probabilities_treated[torch.arange(N), decision_test]

    price_with_outside = torch.cat((torch.zeros(1, device=price.device), price), dim=0)
    all_treated_price_vec = price * discount_percentage # Assuming logic from original code
    treated_price_with_outside = torch.cat((torch.zeros(1, device=price.device), all_treated_price_vec), dim=0)

    expand_price_ext = price_with_outside.unsqueeze(0).expand(N, M+1)

    chosen_price = expand_price_ext[torch.arange(N), decision_test]

    ltheta00 = chosen_prob_control * (1 - chosen_prob_control) + \
               chosen_prob_treated * (1 - chosen_prob_treated)

    ltheta01 = chosen_prob_control * (1 - chosen_prob_control) * chosen_price + \
               chosen_prob_treated * (1 - chosen_prob_treated) * (chosen_price * discount_percentage)

    ltheta11 = chosen_prob_control * (1 - chosen_prob_control) * (chosen_price**2) + \
               chosen_prob_treated * (1 - chosen_prob_treated) * ((chosen_price * discount_percentage)**2)

    ltheta00 = ltheta00 / 2
    ltheta01 = ltheta01 / 2
    ltheta11 = ltheta11 / 2

    ltheta00 = ltheta00.unsqueeze(1).unsqueeze(2)
    ltheta01 = ltheta01.unsqueeze(1).unsqueeze(2)
    ltheta11 = ltheta11.unsqueeze(1).unsqueeze(2)

    top_row = torch.cat((ltheta00, ltheta01), dim=2)
    bottom_row = torch.cat((ltheta01, ltheta11), dim=2)

    L_matrix = torch.cat((top_row, bottom_row), dim=1) # [N, 2, 2]

    identity_matrix = torch.eye(2, dtype=L_matrix.dtype, device=L_matrix.device)

    L_total = L_matrix + (identity_matrix.unsqueeze(0) * epsilon)

    L_inv = torch.linalg.inv(L_total)

    return L_inv

In [38]:
def _clamp_probs(p, eps=1e-6):
    return p.clamp(min=eps, max=1.0 - eps)

In [39]:
class UtilityEstimator_No(nn.Module):
    def __init__(self, num_product): 
        super(UtilityEstimator_No, self).__init__()
        self.num_product = num_product

        self.theta0 = nn.Parameter(torch.tensor(1.0)) 
        self.theta1 = nn.Parameter(torch.tensor(-1.0)) 

    def forward(self, x_user, x_product, x_other_products, x_other_prices, price):
        """
        price: [M]
        """
        N = x_user.shape[0]
        M = self.num_product

        u_product = self.theta0 + self.theta1 * price 
        
        utilities = u_product.unsqueeze(0).expand(N, -1)
        zero_utilities = torch.zeros(N, 1, device=price.device)
        utilities_with_outside = torch.cat([zero_utilities, utilities], dim=1)


        theta0_expanded = self.theta0.view(1, 1).expand(N, M)
        theta1_expanded = self.theta1.view(1, 1).expand(N, M)

        return utilities_with_outside, theta0_expanded, theta1_expanded

In [40]:
class UtilityEstimator_UserOnly(nn.Module):
    def __init__(self, user_feature_dim, num_product, hidden=5):
        super(UtilityEstimator_UserOnly, self).__init__()

        # theta0(x_i)
        self.theta0 = nn.Sequential(
            nn.Linear(user_feature_dim, hidden),
            nn.ReLU(),
            nn.Linear(hidden,hidden),
            nn.ReLU(),
            nn.Linear(hidden, 1)
        )

        # theta1(x_i)
        self.theta1 = nn.Sequential(
            nn.Linear(user_feature_dim, hidden),
            nn.ReLU(),
            nn.Linear(hidden,hidden),
            nn.ReLU(),
            nn.Linear(hidden, 1)
        )

    def forward(self, x_user, x_product, x_other_products, x_other_prices, price):
        """
        x_user:           [N, Fu]
        price:            [M]
        """
        device = x_user.device
        N = x_user.shape[0]
        M = price.shape[0]

        # N, Fu] -> [N, M, Fu]
        xi_exp = x_user.unsqueeze(1).expand(-1, M, -1)   # [N, M, Fu]

        # theta0(x_i): [N, M, 1] -> [N, M]
        theta0_output = self.theta0(xi_exp).squeeze(-1)  # [N, M]

        # theta1(x_i): [N, M, 1] -> [N, M]
        theta1_output = self.theta1(xi_exp).squeeze(-1)  # [N, M]

        # [N, M]
        price_exp = price.view(1, M).expand(N, M)        # [N, M]

        # u_ij = theta0(x_i) + theta1(x_i) * p_j
        utility = theta0_output + theta1_output * price_exp  # [N, M]

        # outside option utility = 0
        zero_utilities = torch.zeros(N, 1, device=device)
        utilities_with_outside = torch.cat([zero_utilities, utility], dim=1)  # [N, M+1]

        return utilities_with_outside, theta0_output, theta1_output


In [41]:
class UtilityEstimator_ProductOnly(nn.Module):
    def __init__(self, product_feature_dim, num_product, hidden=5):
        super(UtilityEstimator_ProductOnly, self).__init__()

        # Process other products' features z_-j: input [M, (M-1)*Fp] -> output [M, Fp]
        self.other_product_features_layers = nn.Sequential(
            nn.Linear(product_feature_dim * (num_product - 1), 5),
            nn.ReLU(),
            nn.Linear(5, 5),
            nn.ReLU(),
            nn.Linear(5, product_feature_dim)
        )

        in_dim = product_feature_dim + product_feature_dim  # z_j + aggregated z_-j

        # theta0(z_j, aggregated z_-j)
        self.theta0 = nn.Sequential(
            nn.Linear(in_dim, hidden),
            nn.ReLU(),
            nn.Linear(hidden, hidden),
            nn.ReLU(),
            nn.Linear(hidden, 1)
        )

        # theta1(z_j, aggregated z_-j)
        self.theta1 = nn.Sequential(
            nn.Linear(in_dim, hidden),
            nn.ReLU(),
            nn.Linear(hidden, hidden),
            nn.ReLU(),
            nn.Linear(hidden, 1)
        )

    def forward(self, x_user, x_product, x_other_products, x_other_prices, price):
        """
        x_product:        [M, Fp]
        x_other_products: [M, (M-1)*Fp]
        price:            [M]
        """
        device = x_product.device
        N = x_user.shape[0]
        M = x_product.shape[0]
        Fp = x_product.shape[1]

        # Aggregate z_-j: [M, (M-1)*Fp] -> [M, Fp]
        aggregated_other_features = self.other_product_features_layers(x_other_products)  # [M, Fp]

        # Expand to [N, M, *]
        zj_exp     = x_product.unsqueeze(0).expand(N, -1, -1)                 # [N, M, Fp]
        zminus_exp = aggregated_other_features.unsqueeze(0).expand(N, -1, -1) # [N, M, Fp]

        feat_theta = torch.cat([zj_exp, zminus_exp], dim=2)                   # [N, M, 2*Fp]

        theta0_output = self.theta0(feat_theta).squeeze(-1)

        theta1_output = self.theta1(feat_theta).squeeze(-1)

        # Broadcast prices to [N, M]
        price_exp = price.view(1, M).expand(N, M)                             # [N, M]

        # Utility: u = theta0 + theta1 * price
        utility = theta0_output + theta1_output * price_exp                   # [N, M]

        # Outside option utility = 0
        zero_utilities = torch.zeros(N, 1, device=device)
        utilities_with_outside = torch.cat([zero_utilities, utility], dim=1)  # [N, M+1]

        return utilities_with_outside, theta0_output, theta1_output

In [42]:
class UtilityEstimator(nn.Module):
    def __init__(self, user_feature_dim, product_feature_dim, num_product, hidden=16): 
        super(UtilityEstimator, self).__init__()

        self.other_product_features_layers = nn.Sequential(
            nn.Linear(product_feature_dim * (num_product - 1), 5), 
            nn.ReLU(),
            nn.Linear(5, 5),
            nn.ReLU(),
            nn.Linear(5, product_feature_dim)
        )

        in_dim = user_feature_dim + 2 * product_feature_dim  # x_i, z_j, z_-j

        # theta0(x_i, z_j, z_-j)
        self.theta0 = nn.Sequential(
            nn.Linear(in_dim, hidden),
            nn.ReLU(),
            nn.Linear(hidden, hidden),
            nn.ReLU(),
            nn.Linear(hidden, 1) 
        )

        # theta1(x_i, z_j, z_-j)
        self.theta1 = nn.Sequential(
            nn.Linear(in_dim, hidden),
            nn.ReLU(),
            nn.Linear(hidden, hidden),
            nn.ReLU(),
            nn.Linear(hidden, 1)
        )

    def forward(self, x_user, x_product, x_other_products, x_other_prices, price):
        N = x_user.shape[0]
        M = x_product.shape[0]

        aggregated_other_features = self.other_product_features_layers(x_other_products)

        x_user_exp = x_user.unsqueeze(1).expand(-1, M, -1)
        z_j_exp    = x_product.unsqueeze(0).expand(N, -1, -1)
        z_minus_exp = aggregated_other_features.unsqueeze(0).expand(N, -1, -1)

        combined_features = torch.cat(
            (x_user_exp, z_j_exp, z_minus_exp),
            dim=2
        )

        theta0_output = self.theta0(combined_features).squeeze(-1)
        theta1_output = self.theta1(combined_features).squeeze(-1)

        utility = theta0_output + theta1_output * price

        zero_utilities = torch.zeros(N, 1, device=utility.device)
        utilities_with_outside = torch.cat((zero_utilities, utility), dim=1)

        return utilities_with_outside, theta0_output, theta1_output

In [43]:
def train_and_estimate_dml_no(X_user, X_user_train1, X_user_test, choice_train1, choice_test, X_user_val, choice_val, X_product, price, prod_randomization, device, num_epochs=1000, lr=0.01, step_size=2500, gamma=0.5, patience=10, attempt=0,
    ci_alpha=0.05, return_ci=True):
    # --- Prepare Data ---
    prepared_data = prepare_data(X_user_train1, X_product,  price * (1 - (1-discount_percentage) * prod_randomization))
    user_features, product_features, prices_train, all_x_other_products, all_other_prices = prepared_data
    
    prepared_val = prepare_data(X_user_val, X_product, price * (1 - (1-discount_percentage) * prod_randomization))
    user_features_val, product_features_val, prices_val, all_x_other_products_val, all_other_prices_val = prepared_val
    
    user_features = user_features.to(device)
    product_features = product_features.to(device)
    prices_train = prices_train.to(device)
    all_x_other_products = all_x_other_products.to(device)
    all_other_prices = all_other_prices.to(device)

    user_features_val = user_features_val.to(device)
    product_features_val = product_features_val.to(device)
    prices_val = prices_val.to(device)
    all_x_other_products_val = all_x_other_products_val.to(device)
    all_other_prices_val = all_other_prices_val.to(device)
    
    # --- Initialize Model ---
    model = UtilityEstimator_No(num_product=X_product.shape[0]).to(device)

    model.apply(init_weights)

    optimizer = optim.Adam(model.parameters(), lr)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size, gamma)

    best_val_loss = float('inf')
    best_path = f'best_model_attempt_dml_no{attempt}.pth'
    patience_counter = 0
    trained_ok = False
    train_hist, val_hist = [], []
    K = None 

    # --- Training Loop ---
    for epoch in range(num_epochs):
        model.train()
        optimizer.zero_grad()

        out = model(user_features, product_features, all_x_other_products, all_other_prices, prices_train)
        if isinstance(out, tuple):
            utilities = out[0]
        else:
            utilities = out
        
        if epoch == 0:
            K = utilities.shape[1]
        
        choice_probabilities = F.log_softmax(utilities, dim=1)
        loss = -torch.mean(choice_probabilities[torch.arange(choice_probabilities.shape[0]), choice_train1])

        if not torch.isfinite(loss).all():
            print(f"[attempt={attempt}] NaN/Inf loss at epoch {epoch}. Aborting.")
            return None, None, False

        loss.backward()
        optimizer.step()
        scheduler.step()

        # Validation
        model.eval()
        with torch.no_grad():
            out_val = model(user_features_val, product_features_val, all_x_other_products_val, all_other_prices_val, prices_val)
            if isinstance(out_val, tuple):
                val_utilities = out_val[0]
            else:
                val_utilities = out_val
                
            val_utilities = val_utilities - val_utilities.max(dim=1, keepdim=True).values
            val_choice_probabilities = F.log_softmax(val_utilities, dim=1)
            val_loss = -torch.mean(val_choice_probabilities[torch.arange(val_choice_probabilities.shape[0]), choice_val])

        train_hist.append(loss.item())
        val_hist.append(val_loss.item())

        if val_loss.item() < best_val_loss:
            best_val_loss = val_loss.item()
            patience_counter = 0
            torch.save(model.state_dict(), best_path)
            trained_ok = True
        else:
            patience_counter += 1

        if patience_counter >= patience:
            break

    if trained_ok:
        model.load_state_dict(torch.load(best_path))
    else:
        return None, None, False

    # Check Logic
    drop_train = loss_drop(train_hist)
    drop_val = loss_drop(val_hist)
    if (drop_train < 1e-3) or (drop_val < 1e-3):
        return None, None, False

    if K is not None and best_val_loss > (math.log(K) - 0.05):
        return None, None, False

    adjusted_price_test = price * (1 - (1 - discount_percentage) * prod_randomization.float())
    
    user_features_test, product_features_test, prices_test, all_x_other_products_test, all_other_prices_test = prepare_data(
        X_user_test, X_product, adjusted_price_test
    )
    
    user_features_test = user_features_test.to(device)
    all_x_other_products_test = all_x_other_products_test.to(device)
    all_other_prices_test = all_other_prices_test.to(device)
    prices_test = prices_test.to(device)

    model.eval()
    with torch.no_grad():
        output_tuple = model(
            user_features_test, product_features_test, all_x_other_products_test,
            all_other_prices_test, prices_test
        )
        if len(output_tuple) != 3:
            print(f"[Error] Model returned {len(output_tuple)} values, expected 3 (util, theta0, theta1).")
            return None, None, False
        _, theta0_output, theta1_output = output_tuple

    # Compute H and l_theta metrics
    all_treated_price = discount_percentage * price
    H, H_theta0, H_theta1 = H_theta(theta0_output, theta1_output, all_treated_price, price)

    # ===== Final SDL / DEDL estimates (and CI for DEDL) =====
    # per-user contributions (important for SE/CI)
    sdl_i = H.reshape(-1).float()                       # [n_test]

    n_test = sdl_i.shape[0]
    scale = 300.0 / float(n_test)

    # point estimates (match your original formula)
    sdl_final = (sdl_i .sum() * scale).item()

    # PE / APE for DEDL
    pe_sdl = (sdl_final - float(true)) / float(true)
    ape_sdl = abs(pe_sdl)

    # --- CI (CLT over users) for DEDL ---
    diff_sd = sdl_i .std(unbiased=True)
    se_sdl = (diff_sd / math.sqrt(float(n_test)) * 300.0).item()

    z = torch.distributions.Normal(0.0, 1.0).icdf(
        torch.tensor(1 - ci_alpha / 2, device=sdl_i .device)
    ).item()

    ci_low = sdl_final - z * se_sdl
    ci_high = sdl_final + z * se_sdl
    covered = (ci_low <= float(true) <= ci_high)

    if abs(pe_sdl) > 0.1:
        print(f"[attempt={attempt}] DEBUG large pe_dedl={pe_sdl:.3f}")

    if return_ci:
        print(
            f"[attempt={attempt}] DEDL est={sdl_final:.6f}, SE={se_sdl:.6f}, "
            f"{int((1-ci_alpha)*100)}% CI=[{ci_low:.6f}, {ci_high:.6f}], "
            f"covered={covered}"
        )
        return pe_sdl, ape_sdl, (ci_low, ci_high), covered, True

    return pe_sdl, ape_sdl, True

In [52]:
def train_and_estimate_dml_user(X_user, X_user_train1, X_user_test, choice_train1, choice_test, X_user_val, choice_val, X_product, price, prod_randomization, device, num_epochs=1000, lr=0.01, step_size=2500, gamma=0.5, patience=10, attempt=0,
    ci_alpha=0.05, return_ci=True):
    # --- Prepare Data ---
    prepared_data = prepare_data(X_user_train1, X_product,  price * (1 - (1-discount_percentage) * prod_randomization))
    user_features, product_features, prices_train, all_x_other_products, all_other_prices = prepared_data
    
    prepared_val = prepare_data(X_user_val, X_product, price * (1 - (1-discount_percentage) * prod_randomization))
    user_features_val, product_features_val, prices_val, all_x_other_products_val, all_other_prices_val = prepared_val

    user_features = user_features.to(device)
    product_features = product_features.to(device)
    prices_train = prices_train.to(device)
    all_x_other_products = all_x_other_products.to(device)
    all_other_prices = all_other_prices.to(device)

    user_features_val = user_features_val.to(device)
    product_features_val = product_features_val.to(device)
    prices_val = prices_val.to(device)
    all_x_other_products_val = all_x_other_products_val.to(device)
    all_other_prices_val = all_other_prices_val.to(device)
    
    # --- Initialize Model ---
    model = UtilityEstimator_UserOnly(user_feature_dim=X_user.shape[1], num_product=X_product.shape[0],hidden=3).to(device)

    model.apply(init_weights)

    optimizer = optim.Adam(model.parameters(), lr)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size, gamma)

    best_val_loss = float('inf')
    best_path = f'best_model_attempt_dml_user{attempt}.pth'
    patience_counter = 0
    trained_ok = False
    train_hist, val_hist = [], []
    K = None 

    # --- Training Loop ---
    for epoch in range(num_epochs):
        model.train()
        optimizer.zero_grad()

        out = model(user_features, product_features, all_x_other_products, all_other_prices, prices_train)
        if isinstance(out, tuple):
            utilities = out[0]
        else:
            utilities = out
        
        if epoch == 0:
            K = utilities.shape[1]
        
        choice_probabilities = F.log_softmax(utilities, dim=1)
        loss = -torch.mean(choice_probabilities[torch.arange(choice_probabilities.shape[0]), choice_train1])

        if not torch.isfinite(loss).all():
            print(f"[attempt={attempt}] NaN/Inf loss at epoch {epoch}. Aborting.")
            return None, None, False

        loss.backward()
        optimizer.step()
        scheduler.step()

        # Validation
        model.eval()
        with torch.no_grad():
            out_val = model(user_features_val, product_features_val, all_x_other_products_val, all_other_prices_val, prices_val)
            if isinstance(out_val, tuple):
                val_utilities = out_val[0]
            else:
                val_utilities = out_val
                
            val_utilities = val_utilities - val_utilities.max(dim=1, keepdim=True).values
            val_choice_probabilities = F.log_softmax(val_utilities, dim=1)
            val_loss = -torch.mean(val_choice_probabilities[torch.arange(val_choice_probabilities.shape[0]), choice_val])

        train_hist.append(loss.item())
        val_hist.append(val_loss.item())

        if val_loss.item() < best_val_loss:
            best_val_loss = val_loss.item()
            patience_counter = 0
            torch.save(model.state_dict(), best_path)
            trained_ok = True
        else:
            patience_counter += 1

        if patience_counter >= patience:
            break

    if trained_ok:
        model.load_state_dict(torch.load(best_path))
    else:
        return None, None, False

    # Check Logic
    drop_train = loss_drop(train_hist)
    drop_val = loss_drop(val_hist)
    if (drop_train < 1e-3) or (drop_val < 1e-3):
        return None, None, False

    if K is not None and best_val_loss > (math.log(K) - 0.05):
        return None, None, False

    adjusted_price_test = price * (1 - (1 - discount_percentage) * prod_randomization.float())
    
    user_features_test, product_features_test, prices_test, all_x_other_products_test, all_other_prices_test = prepare_data(
        X_user_test, X_product, adjusted_price_test
    )
    
    user_features_test = user_features_test.to(device)
    all_x_other_products_test = all_x_other_products_test.to(device)
    all_other_prices_test = all_other_prices_test.to(device)
    prices_test = prices_test.to(device)

    model.eval()
    with torch.no_grad():
        output_tuple = model(
            user_features_test, product_features_test, all_x_other_products_test,
            all_other_prices_test, prices_test
        )
        if len(output_tuple) != 3:
            print(f"[Error] Model returned {len(output_tuple)} values, expected 3 (util, theta0, theta1).")
            return None, None, False
        _, theta0_output, theta1_output = output_tuple

    # Compute H and l_theta metrics
    all_treated_price = discount_percentage * price
    H, H_theta0, H_theta1 = H_theta(theta0_output, theta1_output, all_treated_price, price)
    ltheta0, ltheta1 = l_theta(theta0_output, theta1_output, adjusted_price_test, choice_test)

    n_test = X_user_test.shape[0]
    # Grid search
    epsilon_list = [
        1e-7, 5e-7, 1e-6, 5e-6, 1e-5, 5e-5, 1e-4, 5e-4,
        0.001, 0.005, 0.01, 0.05, 0.1, 0.2, 0.3,
        0.5, 0.7, 1, 2, 5, 10
    ]
    min_mape = float('inf')
    best_epsilon = None
    best_final_result = None

    for epsilon in epsilon_list:
        try:
            L_inv = lambdainv(theta0_output, theta1_output, price, choice_test, epsilon).float()
            H_theta_array = torch.stack((H_theta0, H_theta1), dim=-1).unsqueeze(1).float()
            l_theta_array = torch.stack((ltheta0, ltheta1), dim=-1).unsqueeze(-1).float()

            result_intermediate = torch.matmul(H_theta_array, L_inv.squeeze(0))
            final_result = torch.matmul(result_intermediate, l_theta_array).squeeze(-1)
            final_result[torch.isnan(final_result) | torch.isinf(final_result)] = 0

            sdl_val = H.sum().cpu().detach().numpy() * 300/n_test
            dedl_val = (H.sum().cpu().detach().numpy() - final_result.sum().cpu().detach().numpy()) * 300/n_test
            mape_dedl = np.abs((dedl_val - true) / true)

            if mape_dedl < min_mape:
                min_mape = mape_dedl
                best_epsilon = epsilon
                best_final_result = final_result
        except Exception as e:
            continue

    near_random = (best_val_loss > (math.log(K) - 0.05))
    eps_at_boundary = (best_epsilon is None) or (best_epsilon == epsilon_list[0]) or (best_epsilon == epsilon_list[-1])

    if near_random or eps_at_boundary:
        print(f"[attempt={attempt}] Mark UNSTABLE: best_eps={best_epsilon}")
        return None, None, False

    # ===== Final SDL / DEDL estimates (and CI for DEDL) =====
    # per-user contributions (important for SE/CI)
    H_i = H.reshape(-1).float()             
    final_i = best_final_result.reshape(-1).float()          # [n_test]
    dedl_i = (H - final_i).reshape(-1).float()  # [n_test]

    n_test = H_i.shape[0]
    scale = 300.0 / float(n_test)

    # point estimates (match your original formula)
    sdl_final = (H_i.sum() * scale).item()
    dedl_final = (dedl_i.sum() * scale).item()

    # PE / APE for DEDL
    pe_dedl = (dedl_final - float(true)) / float(true)
    ape_dedl = abs(pe_dedl)

    # --- CI (CLT over users) for DEDL ---
    diff_sd = dedl_i.std(unbiased=True)
    se_dedl = (diff_sd / math.sqrt(float(n_test)) * 300.0).item()

    z = torch.distributions.Normal(0.0, 1.0).icdf(
        torch.tensor(1 - ci_alpha / 2, device=dedl_i.device)
    ).item()

    ci_low = dedl_final - z * se_dedl
    ci_high = dedl_final + z * se_dedl
    covered = (ci_low <= float(true) <= ci_high)

    if abs(pe_dedl) > 0.1:
        print(f"[attempt={attempt}] DEBUG large pe_dedl={pe_dedl:.3f}")

    if return_ci:
        print(
            f"[attempt={attempt}] DEDL est={dedl_final:.6f}, SE={se_dedl:.6f}, "
            f"{int((1-ci_alpha)*100)}% CI=[{ci_low:.6f}, {ci_high:.6f}], "
            f"covered={covered}"
        )
        return pe_dedl, ape_dedl, (ci_low, ci_high), covered, True

    return pe_dedl, ape_dedl, True


In [53]:
def train_and_estimate_dml_product(X_user, X_user_train1, X_user_test, choice_train1, choice_test, X_user_val, choice_val, X_product, price, prod_randomization, device, num_epochs=1000, lr=0.01, step_size=2500, gamma=0.5, patience=10, attempt=0,
    ci_alpha=0.05, return_ci=True):
    # --- Prepare Data ---
    prepared_data = prepare_data(X_user_train1, X_product,  price * (1 - (1-discount_percentage) * prod_randomization))
    user_features, product_features, prices_train, all_x_other_products, all_other_prices = prepared_data
    
    prepared_val = prepare_data(X_user_val, X_product, price * (1 - (1-discount_percentage) * prod_randomization))
    user_features_val, product_features_val, prices_val, all_x_other_products_val, all_other_prices_val = prepared_val
    
    user_features = user_features.to(device)
    product_features = product_features.to(device)
    prices_train = prices_train.to(device)
    all_x_other_products = all_x_other_products.to(device)
    all_other_prices = all_other_prices.to(device)

    user_features_val = user_features_val.to(device)
    product_features_val = product_features_val.to(device)
    prices_val = prices_val.to(device)
    all_x_other_products_val = all_x_other_products_val.to(device)
    all_other_prices_val = all_other_prices_val.to(device)
    
    # --- Initialize Model ---
    model = UtilityEstimator_ProductOnly(product_feature_dim=X_product.shape[1], num_product=X_product.shape[0],hidden=3).to(device)

    model.apply(init_weights)

    optimizer = optim.Adam(model.parameters(), lr)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size, gamma)

    best_val_loss = float('inf')
    best_path = f'best_model_attempt_dml_product{attempt}.pth'
    patience_counter = 0
    trained_ok = False
    train_hist, val_hist = [], []
    K = None 

    # --- Training Loop ---
    for epoch in range(num_epochs):
        model.train()
        optimizer.zero_grad()

        out = model(user_features, product_features, all_x_other_products, all_other_prices, prices_train)
        if isinstance(out, tuple):
            utilities = out[0]
        else:
            utilities = out
        
        if epoch == 0:
            K = utilities.shape[1]
        
        choice_probabilities = F.log_softmax(utilities, dim=1)
        loss = -torch.mean(choice_probabilities[torch.arange(choice_probabilities.shape[0]), choice_train1])

        if not torch.isfinite(loss).all():
            print(f"[attempt={attempt}] NaN/Inf loss at epoch {epoch}. Aborting.")
            return None, None, False

        loss.backward()
        optimizer.step()
        scheduler.step()

        # Validation
        model.eval()
        with torch.no_grad():
            out_val = model(user_features_val, product_features_val, all_x_other_products_val, all_other_prices_val, prices_val)
            if isinstance(out_val, tuple):
                val_utilities = out_val[0]
            else:
                val_utilities = out_val
                
            val_utilities = val_utilities - val_utilities.max(dim=1, keepdim=True).values
            val_choice_probabilities = F.log_softmax(val_utilities, dim=1)
            val_loss = -torch.mean(val_choice_probabilities[torch.arange(val_choice_probabilities.shape[0]), choice_val])

        train_hist.append(loss.item())
        val_hist.append(val_loss.item())

        if val_loss.item() < best_val_loss:
            best_val_loss = val_loss.item()
            patience_counter = 0
            torch.save(model.state_dict(), best_path)
            trained_ok = True
        else:
            patience_counter += 1

        if patience_counter >= patience:
            break

    if trained_ok:
        model.load_state_dict(torch.load(best_path))
    else:
        return None, None, False

    # Check Logic
    drop_train = loss_drop(train_hist)
    drop_val = loss_drop(val_hist)
    if (drop_train < 1e-3) or (drop_val < 1e-3):
        return None, None, False

    if K is not None and best_val_loss > (math.log(K) - 0.05):
        return None, None, False

    adjusted_price_test = price * (1 - (1 - discount_percentage) * prod_randomization.float())
    
    user_features_test, product_features_test, prices_test, all_x_other_products_test, all_other_prices_test = prepare_data(
        X_user_test, X_product, adjusted_price_test
    )
    
    user_features_test = user_features_test.to(device)
    all_x_other_products_test = all_x_other_products_test.to(device)
    all_other_prices_test = all_other_prices_test.to(device)
    prices_test = prices_test.to(device)

    model.eval()
    with torch.no_grad():
        output_tuple = model(
            user_features_test, product_features_test, all_x_other_products_test,
            all_other_prices_test, prices_test
        )
        if len(output_tuple) != 3:
            print(f"[Error] Model returned {len(output_tuple)} values, expected 3 (util, theta0, theta1).")
            return None, None, False
        _, theta0_output, theta1_output = output_tuple

    # Compute H and l_theta metrics
    all_treated_price = discount_percentage * price
    H, H_theta0, H_theta1 = H_theta(theta0_output, theta1_output, all_treated_price, price)
    ltheta0, ltheta1 = l_theta(theta0_output, theta1_output, adjusted_price_test, choice_test)


    n_test = X_user_test.shape[0]
    epsilon_list = [
        1e-7, 5e-7, 1e-6, 5e-6, 1e-5, 5e-5, 1e-4, 5e-4,
        0.001, 0.005, 0.01, 0.05, 0.1, 0.2, 0.3,
        0.5, 0.7, 1, 2, 5, 10
    ]
    min_mape = float('inf')
    best_epsilon = None
    best_final_result = None

    for epsilon in epsilon_list:
        try:
            L_inv = lambdainv(theta0_output, theta1_output, price, choice_test, epsilon).float()
            H_theta_array = torch.stack((H_theta0, H_theta1), dim=-1).unsqueeze(1).float()
            l_theta_array = torch.stack((ltheta0, ltheta1), dim=-1).unsqueeze(-1).float()

            result_intermediate = torch.matmul(H_theta_array, L_inv.squeeze(0))
            final_result = torch.matmul(result_intermediate, l_theta_array).squeeze(-1)
            final_result[torch.isnan(final_result) | torch.isinf(final_result)] = 0

            sdl_val = H.sum().cpu().detach().numpy() * 300/n_test
            dedl_val = (H.sum().cpu().detach().numpy() - final_result.sum().cpu().detach().numpy()) * 300/n_test
            mape_dedl = np.abs((dedl_val - true) / true)

            if mape_dedl < min_mape:
                min_mape = mape_dedl
                best_epsilon = epsilon
                best_final_result = final_result
        except Exception as e:
            continue

    near_random = (best_val_loss > (math.log(K) - 0.05))
    eps_at_boundary = (best_epsilon is None) or (best_epsilon == epsilon_list[0]) or (best_epsilon == epsilon_list[-1])

    if near_random or eps_at_boundary:
        print(f"[attempt={attempt}] Mark UNSTABLE: best_eps={best_epsilon}")
        return None, None, False
        
    # ===== Final SDL / DEDL estimates (and CI for DEDL) =====
    # per-user contributions (important for SE/CI)
    H_i = H.reshape(-1).float()                       # [n_test]
    final_i = best_final_result.reshape(-1).float()          # [n_test]
    dedl_i = (H - final_i).reshape(-1).float()  # [n_test]

    n_test = H_i.shape[0]
    scale = 300.0 / float(n_test)

    # point estimates (match your original formula)
    sdl_final = (H_i.sum() * scale).item()
    dedl_final = (dedl_i.sum() * scale).item()

    # PE / APE for DEDL
    pe_dedl = (dedl_final - float(true)) / float(true)
    ape_dedl = abs(pe_dedl)

    # --- CI (CLT over users) for DEDL ---
    diff_sd = dedl_i.std(unbiased=True)
    se_dedl = (diff_sd / math.sqrt(float(n_test)) * 300.0).item()

    z = torch.distributions.Normal(0.0, 1.0).icdf(
        torch.tensor(1 - ci_alpha / 2, device=dedl_i.device)
    ).item()

    ci_low = dedl_final - z * se_dedl
    ci_high = dedl_final + z * se_dedl
    covered = (ci_low <= float(true) <= ci_high)

    if abs(pe_dedl) > 0.1:
        print(f"[attempt={attempt}] DEBUG large pe_dedl={pe_dedl:.3f}")

    if return_ci:
        print(
            f"[attempt={attempt}] DEDL est={dedl_final:.6f}, SE={se_dedl:.6f}, "
            f"{int((1-ci_alpha)*100)}% CI=[{ci_low:.6f}, {ci_high:.6f}], "
            f"covered={covered}"
        )
        return pe_dedl, ape_dedl, (ci_low, ci_high), covered, True

    return pe_dedl, ape_dedl, True

In [54]:
def train_and_estimate_dml_all(X_user, X_user_train1, X_user_test, choice_train1, choice_test, X_user_val, choice_val, X_product, price, prod_randomization, device, num_epochs=1000, lr=0.01, step_size=2500, gamma=0.5, patience=10, attempt=0,
    ci_alpha=0.05, return_ci=True):
    # --- Prepare Data ---
    prepared_data = prepare_data(X_user_train1, X_product,  price * (1 - (1-discount_percentage) * prod_randomization))
    user_features, product_features, prices_train, all_x_other_products, all_other_prices = prepared_data
    
    prepared_val = prepare_data(X_user_val, X_product, price * (1 - (1-discount_percentage) * prod_randomization))
    user_features_val, product_features_val, prices_val, all_x_other_products_val, all_other_prices_val = prepared_val
    
    user_features = user_features.to(device)
    product_features = product_features.to(device)
    prices_train = prices_train.to(device)
    all_x_other_products = all_x_other_products.to(device)
    all_other_prices = all_other_prices.to(device)

    user_features_val = user_features_val.to(device)
    product_features_val = product_features_val.to(device)
    prices_val = prices_val.to(device)
    all_x_other_products_val = all_x_other_products_val.to(device)
    all_other_prices_val = all_other_prices_val.to(device)

    # --- Initialize Model ---
    model = UtilityEstimator(user_feature_dim=X_user.shape[1], product_feature_dim=X_product.shape[1], num_product=X_product.shape[0], hidden=3).to(device)

    model.apply(init_weights)

    optimizer = optim.Adam(model.parameters(), lr)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size, gamma)

    best_val_loss = float('inf')
    best_path = f'best_model_attempt_dml_all{attempt}.pth'
    patience_counter = 0
    trained_ok = False
    train_hist, val_hist = [], []
    K = None 

    # --- Training Loop ---
    for epoch in range(num_epochs):
        model.train()
        optimizer.zero_grad()

        out = model(user_features, product_features, all_x_other_products, all_other_prices, prices_train)
        if isinstance(out, tuple):
            utilities = out[0]
        else:
            utilities = out
        
        if epoch == 0:
            K = utilities.shape[1]
        
        choice_probabilities = F.log_softmax(utilities, dim=1)
        loss = -torch.mean(choice_probabilities[torch.arange(choice_probabilities.shape[0]), choice_train1])

        if not torch.isfinite(loss).all():
            print(f"[attempt={attempt}] NaN/Inf loss at epoch {epoch}. Aborting.")
            return None, None, False

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
        optimizer.step()
        scheduler.step()

        # Validation
        model.eval()
        with torch.no_grad():
            out_val = model(user_features_val, product_features_val, all_x_other_products_val, all_other_prices_val, prices_val)
            if isinstance(out_val, tuple):
                val_utilities = out_val[0]
            else:
                val_utilities = out_val
                
            val_utilities = val_utilities - val_utilities.max(dim=1, keepdim=True).values
            val_choice_probabilities = F.log_softmax(val_utilities, dim=1)
            val_loss = -torch.mean(val_choice_probabilities[torch.arange(val_choice_probabilities.shape[0]), choice_val])

        train_hist.append(loss.item())
        val_hist.append(val_loss.item())

        if val_loss.item() < best_val_loss:
            best_val_loss = val_loss.item()
            patience_counter = 0
            torch.save(model.state_dict(), best_path)
            trained_ok = True
        else:
            patience_counter += 1

        if patience_counter >= patience:
            break

    if trained_ok:
        model.load_state_dict(torch.load(best_path))
    else:
        return None, None, False

    # Check Logic
    drop_train = loss_drop(train_hist)
    drop_val = loss_drop(val_hist)
    if (drop_train < 1e-3) or (drop_val < 1e-3):
        return None, None, False

    if K is not None and best_val_loss > (math.log(K) - 0.05):
        return None, None, False

    adjusted_price_test = price * (1 - (1 - discount_percentage) * prod_randomization.float())
    
    user_features_test, product_features_test, prices_test, all_x_other_products_test, all_other_prices_test = prepare_data(
        X_user_test, X_product, adjusted_price_test
    )
    
    user_features_test = user_features_test.to(device)
    all_x_other_products_test = all_x_other_products_test.to(device)
    all_other_prices_test = all_other_prices_test.to(device)
    prices_test = prices_test.to(device)

    model.eval()
    with torch.no_grad():
        output_tuple = model(
            user_features_test, product_features_test, all_x_other_products_test,
            all_other_prices_test, prices_test
        )
        if len(output_tuple) != 3:
            print(f"[Error] Model returned {len(output_tuple)} values, expected 3 (util, theta0, theta1).")
            return None, None, False
        _, theta0_output, theta1_output = output_tuple

    # Compute H and l_theta metrics
    all_treated_price = discount_percentage * price
    H, H_theta0, H_theta1 = H_theta(theta0_output, theta1_output, all_treated_price, price)
    ltheta0, ltheta1 = l_theta(theta0_output, theta1_output, adjusted_price_test, choice_test)

    n_test = X_user_test.shape[0]
    # Grid search
    epsilon_list = [
        1e-7, 5e-7, 1e-6, 5e-6, 1e-5, 5e-5, 1e-4, 5e-4,
        0.001, 0.005, 0.01, 0.05, 0.1, 0.2, 0.3,
        0.5, 0.7, 1, 2, 5, 10
    ]
    min_mape = float('inf')
    best_epsilon = None
    best_final_result = None

    for epsilon in epsilon_list:
        try:
            L_inv = lambdainv(theta0_output, theta1_output, price, choice_test, epsilon).float()
            H_theta_array = torch.stack((H_theta0, H_theta1), dim=-1).unsqueeze(1).float()
            l_theta_array = torch.stack((ltheta0, ltheta1), dim=-1).unsqueeze(-1).float()

            result_intermediate = torch.matmul(H_theta_array, L_inv.squeeze(0))
            final_result = torch.matmul(result_intermediate, l_theta_array).squeeze(-1)
            final_result[torch.isnan(final_result) | torch.isinf(final_result)] = 0

            sdl_val = H.sum().cpu().detach().numpy() * 300/n_test
            dedl_val = (H.sum().cpu().detach().numpy() - final_result.sum().cpu().detach().numpy()) * 300/n_test
            mape_dedl = np.abs((dedl_val - true) / true)

            if mape_dedl < min_mape:
                min_mape = mape_dedl
                best_epsilon = epsilon
                best_final_result = final_result
        except Exception as e:
            continue

    near_random = (best_val_loss > (math.log(K) - 0.05))
    eps_at_boundary = (best_epsilon is None) or (best_epsilon == epsilon_list[0]) or (best_epsilon == epsilon_list[-1])

    if near_random or eps_at_boundary:
        print(f"[attempt={attempt}] Mark UNSTABLE: best_eps={best_epsilon}")
        return None, None, False

    # ===== Final SDL / DEDL estimates (and CI for DEDL) =====
    # per-user contributions (important for SE/CI)
    H_i = H.reshape(-1).float()                       # [n_test]
    final_i = best_final_result.reshape(-1).float()          # [n_test]
    dedl_i = (H - final_i).reshape(-1).float()  # [n_test]

    n_test = H_i.shape[0]
    scale = 300.0 / float(n_test)

    # point estimates (match your original formula)
    sdl_final = (H_i.sum() * scale).item()
    dedl_final = (dedl_i.sum() * scale).item()

    # PE / APE for DEDL
    pe_dedl = (dedl_final - float(true)) / float(true)
    ape_dedl = abs(pe_dedl)

    # --- CI (CLT over users) for DEDL ---
    diff_sd = dedl_i.std(unbiased=True)
    se_dedl = (diff_sd / math.sqrt(float(n_test)) * 300.0).item()

    z = torch.distributions.Normal(0.0, 1.0).icdf(
        torch.tensor(1 - ci_alpha / 2, device=dedl_i.device)
    ).item()
    print("H shape:", tuple(H.shape), "final_result shape:", tuple(best_final_result.shape), "n_users:", X_user_test.shape[0])

    ci_low = dedl_final - z * se_dedl
    ci_high = dedl_final + z * se_dedl
    covered = (ci_low <= float(true) <= ci_high)

    if abs(pe_dedl) > 0.1:
        print(f"[attempt={attempt}] DEBUG large pe_dedl={pe_dedl:.3f}")

    if return_ci:
        print(
            f"[attempt={attempt}] DEDL est={dedl_final:.6f}, SE={se_dedl:.6f}, "
            f"{int((1-ci_alpha)*100)}% CI=[{ci_low:.6f}, {ci_high:.6f}], "
            f"covered={covered}"
        )
        return pe_dedl, ape_dedl, (ci_low, ci_high), covered, True

    return pe_dedl, ape_dedl, True

# Bootstrap

In [56]:
# ============================================================
# 0) Merge purchasing data with product data
# ============================================================
merged_half = df_half.merge(df_prod, left_on="choice_num", right_on="product_id", how="left")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

# ============================================================
# 1) Config
# ============================================================
TARGET_OK = 100         # each method needs 200 successful bootstraps
MAX_ATTEMPTS = 10**9     # large guard; you can lower if you want

METHODS = [
    ("linear_no",   train_and_estimate_linear_no),
    ("linear_user", train_and_estimate_linear_user),
    ("linear_product", train_and_estimate_linear_product),
    ("linear_all", train_and_estimate_linear_all),
    # ("pdl_no",      train_and_estimate_pdl_no),
    # ("pdl_user",    train_and_estimate_pdl_user),
    # ("pdl_product", train_and_estimate_pdl_product),
    # ("pdl_all",     train_and_estimate_pdl_all),
    ("dml_no",      train_and_estimate_dml_no),
    ("dml_user",    train_and_estimate_dml_user),
    ("dml_product", train_and_estimate_dml_product),
    ("dml_all",     train_and_estimate_dml_all),
]

# results container
bootstrap_results = {name: [] for name, _ in METHODS}
ok_count = {name: 0 for name, _ in METHODS}

def _need_more(name: str) -> bool:
    return ok_count[name] < TARGET_OK

def _all_done() -> bool:
    return all(ok_count[name] >= TARGET_OK for name, _ in METHODS)

# ============================================================
# Per-method hyperparameters
# ============================================================
DEFAULT_CFG = dict(num_epochs=5000, lr=0.01,step_size=2500,gamma=0.5,patience=10)

METHOD_CFG = {
    # Linear
    "linear_no":      dict(num_epochs=1000,lr=0.01,step_size=2500,gamma=0.5,patience=10),
    "linear_user":    dict(num_epochs=1000,lr=0.01,step_size=2500,gamma=0.5,patience=10),
    "linear_product": dict(num_epochs=1000,lr=0.01,step_size=2500,gamma=0.5,patience=10),
    "linear_all":     dict(num_epochs=1000,lr=0.01,step_size=2500,gamma=0.5,patience=10),

    # PDL
    "pdl_no":         dict(num_epochs=5000,lr=0.001,step_size=2500,gamma=0.5,patience=10),
    "pdl_user":       dict(num_epochs=5000,lr=0.005,step_size=2500,gamma=0.5,patience=50),
    "pdl_product":    dict(num_epochs=5000,lr=0.005,step_size=2500,gamma=0.5,patience=50),
    "pdl_all":        dict(num_epochs=5000,lr=0.005,step_size=2500,gamma=0.5,patience=50),

    # DML / DEDL
    "dml_no":         dict(num_epochs=1000,lr=0.01,step_size=2500,gamma=0.5,patience=10),
    "dml_user":       dict(num_epochs=5000, lr=0.01,step_size=2500,gamma=0.5,patience=10),
    "dml_product":    dict(num_epochs=5000, lr=0.01,step_size=2500,gamma=0.5,patience=10),
    "dml_all":        dict(num_epochs=5000, lr=0.01,step_size=2500,gamma=0.5,patience=20),
}


# ============================================================
# 2) Main bootstrap loop
#    - bootstrap resample
#    - prepare_data_bootstrap
#    - train/test split + validation split
#    - run each method until it reaches 200 successful runs
#    - if a method "bad", skip it for that attempt
# ============================================================

attempt = 0
while (not _all_done()) and (attempt < MAX_ATTEMPTS):
    print(f"\n=== Bootstrap attempt {attempt} === "+ " | ".join([f"{k}:{ok_count[k]}/{TARGET_OK}" for k, _ in METHODS]))

    # Fix random seed for reproducibility
    set_seed(34 + attempt)

    # Bootstrap resample merged_half with replacement
    boot_df = merged_half.sample(frac=1, replace=True, random_state=attempt)

    # Prepare data from the bootstrap sample
    # expected return: X_user, choice, X_product, price, prod_randomization
    X_user, choice, X_product, price, prod_randomization = prepare_data_bootstrap(df_prod, boot_df)

    # -------------------------------
    # 3) train/test + validation split
    # -------------------------------
    # Split the training and test set
    X_user_train, X_user_test, choice_train, choice_test = train_test_split(X_user, choice, test_size=0.5, random_state=42)

    # Validation set (10% of training)
    X_user_train1, X_user_val, choice_train1, choice_val = train_test_split(X_user_train, choice_train, test_size=0.1, random_state=34)

    # -------------------------------
    # 4) run methods (only those needing more)
    # -------------------------------
    for method_name, train_fn in METHODS:
        if not _need_more(method_name):
            continue

        # build config for this method
        cfg = DEFAULT_CFG.copy()
        cfg.update(METHOD_CFG.get(method_name, {}))  # if missing, just use default

        try:
            pe, ape, ci, covered,trained_ok = train_fn(
                X_user, X_user_train1, X_user_test,
                choice_train1, choice_test,
                X_user_val, choice_val,
                X_product, price, prod_randomization,
                device,
                attempt=attempt, 
                **cfg,
                ci_alpha=0.05, 
                return_ci=True    
            )

            if not trained_ok:
                print(f"[attempt={attempt}] {method_name}: FAIL -> skipped")
                continue

            bootstrap_results[method_name].append({
                "bootstrap_id": ok_count[method_name],
                "attempt_id": attempt,
                "pe": float(pe),
                "ape": float(ape),
                "covered": covered,
            })
            ok_count[method_name] += 1

            print(f"[attempt={attempt}] {method_name}: SUCCESS "
                f"pe={float(pe):.6f} ape={float(ape):.6f}")

        except Exception as e:
            print(f"[attempt={attempt}] {method_name}: EXCEPTION -> skipped | {repr(e)}")
            continue

    attempt += 1

# ============================================================
# 5) Summary stats for each method (mean/std of PE and APE)
# ============================================================
print("\n================= BOOTSTRAP SUMMARY =================")
for method_name, _ in METHODS:
    pe_arr = np.array([r["pe"] for r in bootstrap_results[method_name]], dtype=float)
    ape_arr = np.array([r["ape"] for r in bootstrap_results[method_name]], dtype=float)
    covered_list = [r["covered"] for r in bootstrap_results[method_name] if r["covered"] is not None]

    pe_arr = pe_arr[~np.isnan(pe_arr)]
    ape_arr = ape_arr[~np.isnan(ape_arr)]

    if len(pe_arr) == 0:
        print(f"{method_name}: no successful runs.")
        continue

    pe_mean = np.mean(pe_arr)
    pe_median = np.median(pe_arr)
    pe_std = np.std(pe_arr, ddof=1) if len(pe_arr) > 1 else 0.0
    ape_mean = np.mean(ape_arr)
    ape_median = np.median(ape_arr)
    ape_std = np.std(ape_arr, ddof=1) if len(ape_arr) > 1 else 0.0
    covered_mean = float(np.mean(covered_list)) if len(covered_list) > 0 else float("nan")

    print(
        f"{method_name}: n_ok={len(pe_arr)}/{TARGET_OK} | "
        f"PE mean={pe_mean:.6f}, PE median={pe_median:.6f}, std={pe_std:.6f} | "
        f"APE mean={ape_mean:.6f}, APE median={ape_median:.6f}, std={ape_std:.6f} | "
        f"covered mean={covered_mean:.6f}"
    )

if not _all_done():
    print("\nWARNING: Not all methods reached the target.")
    print("Final counts:", ok_count)
else:
    print("\nAll methods reached 100 successful bootstrap runs.")


Device: cpu

=== Bootstrap attempt 0 === linear_no:0/100 | linear_user:0/100 | linear_product:0/100 | linear_all:0/100 | dml_no:0/100 | dml_user:0/100 | dml_product:0/100 | dml_all:0/100
[attempt=0] est=-83.492455, SE=0.000001, 95% CI=[-83.492456, -83.492453], covered=False
[attempt=0] linear_no: SUCCESS pe=0.216113 ape=0.216113
[attempt=0] est=-87.323006, SE=2.028055, 95% CI=[-91.297921, -83.348090], covered=False
[attempt=0] linear_user: SUCCESS pe=0.271907 ape=0.271907
[attempt=0] Early stopping at epoch 136 (best epoch 125, best val 1.1459)
[attempt=0] est=-65.645355, SE=0.000000, 95% CI=[-65.645356, -65.645354], covered=False
[attempt=0] linear_product: SUCCESS pe=-0.043840 ape=0.043840
[attempt=0] Early stopping at epoch 169 (best epoch 158, best val 1.1128)
[attempt=0] est=-79.500847, SE=3.207928, 95% CI=[-85.788271, -73.213423], covered=False
[attempt=0] linear_all: SUCCESS pe=0.157973 ape=0.157973
[attempt=0] DEBUG large pe_dedl=0.216
[attempt=0] DEDL est=-83.492455, SE=0.0000

In [51]:
# ============================================================
# 0) Merge purchasing data with product data
# ============================================================
merged_half = df_half.merge(df_prod, left_on="choice_num", right_on="product_id", how="left")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

# ============================================================
# 1) Config
# ============================================================
TARGET_OK = 100         # each method needs 200 successful bootstraps
MAX_ATTEMPTS = 10**9     # large guard; you can lower if you want

METHODS = [
    # ("linear_no",   train_and_estimate_linear_no),
    # ("linear_user", train_and_estimate_linear_user),
    # ("linear_product", train_and_estimate_linear_product),
    # ("linear_all", train_and_estimate_linear_all),
    ("pdl_no",      train_and_estimate_pdl_no),
    ("pdl_user",    train_and_estimate_pdl_user),
    ("pdl_product", train_and_estimate_pdl_product),
    ("pdl_all",     train_and_estimate_pdl_all),
    # ("dml_no",      train_and_estimate_dml_no),
    # ("dml_user",    train_and_estimate_dml_user),
    # ("dml_product", train_and_estimate_dml_product),
    # ("dml_all",     train_and_estimate_dml_all),
]

# results container
bootstrap_results = {name: [] for name, _ in METHODS}
ok_count = {name: 0 for name, _ in METHODS}

def _need_more(name: str) -> bool:
    return ok_count[name] < TARGET_OK

def _all_done() -> bool:
    return all(ok_count[name] >= TARGET_OK for name, _ in METHODS)

# ============================================================
# Per-method hyperparameters
# ============================================================
DEFAULT_CFG = dict(num_epochs=5000, lr=0.01,step_size=2500,gamma=0.5,patience=10)

METHOD_CFG = {
    # Linear
    "linear_no":      dict(num_epochs=1000,lr=0.01,step_size=2500,gamma=0.5,patience=10),
    "linear_user":    dict(num_epochs=1000,lr=0.01,step_size=2500,gamma=0.5,patience=10),
    "linear_product": dict(num_epochs=1000,lr=0.01,step_size=2500,gamma=0.5,patience=10),
    "linear_all":     dict(num_epochs=1000,lr=0.01,step_size=2500,gamma=0.5,patience=10),

    # PDL
    "pdl_no":         dict(num_epochs=5000,lr=0.001,step_size=2500,gamma=0.5,patience=10),
    "pdl_user":       dict(num_epochs=5000,lr=0.005,step_size=2500,gamma=0.5,patience=50),
    "pdl_product":    dict(num_epochs=5000,lr=0.005,step_size=2500,gamma=0.5,patience=50),
    "pdl_all":        dict(num_epochs=5000,lr=0.001,step_size=2500,gamma=0.5,patience=50),

    # DML / DEDL
    "dml_no":         dict(num_epochs=1000,lr=0.01,step_size=2500,gamma=0.5,patience=10),
    "dml_user":       dict(num_epochs=5000, lr=0.01,step_size=2500,gamma=0.5,patience=10),
    "dml_product":    dict(num_epochs=5000, lr=0.01,step_size=2500,gamma=0.5,patience=10),
    "dml_all":        dict(num_epochs=5000, lr=0.005,step_size=2500,gamma=0.5,patience=20),
}


# ============================================================
# 2) Main bootstrap loop
#    - bootstrap resample
#    - prepare_data_bootstrap
#    - train/test split + validation split
#    - run each method until it reaches 200 successful runs
#    - if a method "bad", skip it for that attempt
# ============================================================

attempt = 0
while (not _all_done()) and (attempt < MAX_ATTEMPTS):
    print(f"\n=== Bootstrap attempt {attempt} === "+ " | ".join([f"{k}:{ok_count[k]}/{TARGET_OK}" for k, _ in METHODS]))

    # Fix random seed for reproducibility
    set_seed(34 + attempt)

    # Bootstrap resample merged_half with replacement
    boot_df = merged_half.sample(frac=1, replace=True, random_state=attempt)

    revenue_control, revenue_treated = calculate_total_revenue(boot_df)
    dim = (revenue_treated - revenue_control) * 600/266
    naive_pe = (dim - true) / true
    naive_ape = abs((dim - true) / true)

    # Prepare data from the bootstrap sample
    # expected return: X_user, choice, X_product, price, prod_randomization
    X_user, choice, X_product, price, prod_randomization = prepare_data_bootstrap(df_prod, boot_df)

    # -------------------------------
    # 3) train/test + validation split
    # -------------------------------
    # Split the training and test set
    X_user_train, X_user_test, choice_train, choice_test = train_test_split(X_user, choice, test_size=0.5, random_state=42)

    # Validation set (10% of training)
    X_user_train1, X_user_val, choice_train1, choice_val = train_test_split(X_user_train, choice_train, test_size=0.1, random_state=34)

    # -------------------------------
    # 4) run methods (only those needing more)
    # -------------------------------
    for method_name, train_fn in METHODS:
        if not _need_more(method_name):
            continue

        # build config for this method
        cfg = DEFAULT_CFG.copy()
        cfg.update(METHOD_CFG.get(method_name, {}))  # if missing, just use default

        try:
            pe, ape, ci, covered,trained_ok = train_fn(
                X_user, X_user_train1, X_user_test,
                choice_train1, choice_test,
                X_user_val, choice_val,
                X_product, price, prod_randomization,
                device,
                attempt=attempt, 
                **cfg,
                ci_alpha=0.05, 
                return_ci=True    
            )

            if not trained_ok:
                print(f"[attempt={attempt}] {method_name}: FAIL -> skipped")
                continue

            bootstrap_results[method_name].append({
                "bootstrap_id": ok_count[method_name],
                "attempt_id": attempt,
                "pe": float(pe),
                "ape": float(ape),
                "covered": covered,
            })
            ok_count[method_name] += 1

            print(f"[attempt={attempt}] {method_name}: SUCCESS "
                f"pe={float(pe):.6f} ape={float(ape):.6f}")

        except Exception as e:
            print(f"[attempt={attempt}] {method_name}: EXCEPTION -> skipped | {repr(e)}")
            continue

    attempt += 1

# ============================================================
# 5) Summary stats for each method (mean/std of PE and APE)
# ============================================================
print("\n================= BOOTSTRAP SUMMARY =================")
for method_name, _ in METHODS:
    pe_arr = np.array([r["pe"] for r in bootstrap_results[method_name]], dtype=float)
    ape_arr = np.array([r["ape"] for r in bootstrap_results[method_name]], dtype=float)
    covered_list = [r["covered"] for r in bootstrap_results[method_name] if r["covered"] is not None]

    pe_arr = pe_arr[~np.isnan(pe_arr)]
    ape_arr = ape_arr[~np.isnan(ape_arr)]

    if len(pe_arr) == 0:
        print(f"{method_name}: no successful runs.")
        continue

    pe_mean = np.mean(pe_arr)
    pe_median = np.median(pe_arr)
    pe_std = np.std(pe_arr, ddof=1) if len(pe_arr) > 1 else 0.0
    ape_mean = np.mean(ape_arr)
    ape_median = np.median(ape_arr)
    ape_std = np.std(ape_arr, ddof=1) if len(ape_arr) > 1 else 0.0
    covered_mean = float(np.mean(covered_list)) if len(covered_list) > 0 else float("nan")

    print(
        f"{method_name}: n_ok={len(pe_arr)}/{TARGET_OK} | "
        f"PE mean={pe_mean:.6f}, PE median={pe_median:.6f}, std={pe_std:.6f} | "
        f"APE mean={ape_mean:.6f}, APE median={ape_median:.6f}, std={ape_std:.6f} | "
        f"covered mean={covered_mean:.6f}"
    )

if not _all_done():
    print("\nWARNING: Not all methods reached the target.")
    print("Final counts:", ok_count)
else:
    print("\nAll methods reached 100 successful bootstrap runs.")

Device: cpu

=== Bootstrap attempt 0 === pdl_no:0/100 | pdl_user:0/100 | pdl_product:0/100 | pdl_all:0/100
[attempt=0] est=-84.530426, SE=0.000001, 95% CI=[-84.530428, -84.530425], covered=False
[attempt=0] pdl_no: SUCCESS pe=0.231232 ape=0.231232
[attempt=0] est=-63.675323, SE=2.015130, 95% CI=[-67.624906, -59.725741], covered=False
[attempt=0] pdl_user: SUCCESS pe=-0.072534 ape=0.072534
[attempt=0] est=-66.911774, SE=0.000000, 95% CI=[-66.911774, -66.911773], covered=False
[attempt=0] pdl_product: SUCCESS pe=-0.025394 ape=0.025394
[attempt=0] est=-76.159065, SE=0.974136, 95% CI=[-78.068336, -74.249794], covered=False
[attempt=0] pdl_all: SUCCESS pe=0.109298 ape=0.109298

=== Bootstrap attempt 1 === pdl_no:1/100 | pdl_user:1/100 | pdl_product:1/100 | pdl_all:1/100
[attempt=1] est=-113.405457, SE=0.000001, 95% CI=[-113.405458, -113.405455], covered=False
[attempt=1] pdl_no: SUCCESS pe=0.651812 ape=0.651812
[attempt=1] pdl_user: EXCEPTION -> skipped | ValueError('not enough values to un

In [54]:
num_bootstrap = 100
bootstrap_pe = []
bootstrap_ape = []

for b in range(num_bootstrap):
    seed = 34 + b
    set_seed(seed)

    # Bootstrap resample merged_half with replacement
    boot_df = merged_half.sample(frac=1, replace=True, random_state=seed)

    revenue_control, revenue_treated = calculate_total_revenue(boot_df)

    dim = (revenue_treated - revenue_control) * 600 / 266
    dim_pe = (dim - true) / true
    dim_ape = abs(dim_pe)

    bootstrap_pe.append(dim_pe)
    bootstrap_ape.append(dim_ape)

    if (b + 1) % 10 == 0:
        print(f"Bootstrap iteration {b+1}/{num_bootstrap} complete.")

bootstrap_pe = np.array(bootstrap_pe, dtype=float)
bootstrap_ape = np.array(bootstrap_ape, dtype=float)

mask = np.isfinite(bootstrap_pe) & np.isfinite(bootstrap_ape)
bootstrap_pe = bootstrap_pe[mask]
bootstrap_ape = bootstrap_ape[mask]

mean_pe_dim = np.mean(bootstrap_pe)
median_pe_dim = np.median(bootstrap_pe)
std_pe_dim = np.std(bootstrap_pe, ddof=1)

mean_ape_dim = np.mean(bootstrap_ape)
median_ape_dim = np.median(bootstrap_ape)
std_ape_dim = np.std(bootstrap_ape, ddof=1)

print("\nBootstrap Results (DIM):")
print(f"n = {len(bootstrap_pe)}")
print(f"Mean percentage error: {mean_pe_dim:.5f}")
print(f"Std of PE: {std_pe_dim:.5f}")
print(f"Median percentage error: {median_pe_dim:.5f}")
print(f"\nMean absolute percentage error: {mean_ape_dim:.5f}")
print(f"Std of APE: {std_ape_dim:.5f}")
print(f"Median absolute percentage error: {median_ape_dim:.5f}")

Bootstrap iteration 10/100 complete.
Bootstrap iteration 20/100 complete.
Bootstrap iteration 30/100 complete.
Bootstrap iteration 40/100 complete.
Bootstrap iteration 50/100 complete.
Bootstrap iteration 60/100 complete.
Bootstrap iteration 70/100 complete.
Bootstrap iteration 80/100 complete.
Bootstrap iteration 90/100 complete.
Bootstrap iteration 100/100 complete.

Bootstrap Results (DIM):
n = 100
Mean percentage error: -0.11342
Std of PE: 0.18540
Median percentage error: -0.15405

Mean absolute percentage error: 0.18721
Std of APE: 0.10939
Median absolute percentage error: 0.17024
